# IF3070 Foundations of Artificial Intelligence | Tugas Besar 2




Group Number: 16

Group Members:
- Thalita Zahra Sutejo (18222023)
- Irfan Musthofa (18222056)
- Eleanor Cordelia (18222059)
- Muhammad Faiz Atharrahman (18222063)

## Import Libraries

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import os
import re
import base64

from urllib.parse import urlparse
from sklearn.model_selection import train_test_split
from scipy.stats import skew
from sklearn.preprocessing import PowerTransformer
from scipy.stats import zscore
from sklearn.metrics import accuracy_score, precision_score, recall_score, classification_report, confusion_matrix
from imblearn.over_sampling import SMOTE
from sklearn.neighbors import KNeighborsClassifier
from sklearn.feature_selection import SelectKBest, chi2, f_classif
from sklearn.metrics import classification_report, accuracy_score, f1_score
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix

## Import Dataset

In [ ]:
# File ID from Google Drive
# https://drive.google.com/file/d/1TcrkpgWV0WshPC48LUguqKSrCHpimCk0/view?usp=sharing => File ID: 1TcrkpgWV0WshPC48LUguqKSrCHpimCk0
# Penggunaan format berikut:
# https://drive.google.com/uc?id={file_id}

# Read File CSV
df = pd.read_csv('https://drive.google.com/uc?id=1pcBlv7jxdaJf3agqnT445vrBmNaav7iL',sep=',', low_memory=False)
df

,id,FILENAME,URL,URLLength,Domain,DomainLength,IsDomainIP,TLD,CharContinuationRate,TLDLegitimateProb,...,Pay,Crypto,HasCopyrightInfo,NoOfImage,NoOfCSS,NoOfJS,NoOfSelfRef,NoOfEmptyRef,NoOfExternalRef,label
0,1,NaN,https://www.northcm.ac.th,24.0,www.northcm.ac.th,17.0,0.0,NaN,0.800000,NaN,...,0.0,0.0,1.0,NaN,3.0,NaN,69.0,NaN,NaN,1
1,4,8135291.txt,http://uqr.to/1il1z,NaN,NaN,NaN,NaN,to,1.000000,0.000896,...,NaN,0.0,0.0,NaN,NaN,NaN,NaN,NaN,1.0,0
2,5,586561.txt,https://www.woolworthsrewards.com.au,35.0,www.woolworthsrewards.com.au,28.0,0.0,au,0.857143,NaN,...,1.0,0.0,1.0,33.0,7.0,8.0,15.0,NaN,2.0,1
3,6,NaN,NaN,31.0,NaN,NaN,NaN,com,0.562500,0.522907,...,1.0,0.0,1.0,24.0,5.0,14.0,NaN,NaN,NaN,1
4,11,412632.txt,NaN,NaN,www.nyprowrestling.com,22.0,0.0,NaN,1.000000,NaN,...,0.0,0.0,1.0,NaN,NaN,14.0,NaN,0.0,NaN,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
140399,235790,49490.txt,NaN,NaN,www.slavevoyages.org,NaN,0.0,NaN,1.000000,0.079963,...,NaN,0.0,1.0,23.0,3.0,6.0,NaN,12.0,NaN,1
140400,235791,NaN,https://www.greenmountainenergy.com,34.0,www.greenmountainenergy.com,NaN,0.0,com,1.000000,0.522907,...,1.0,NaN,1.0,26.0,NaN,NaN,169.0,15.0,40.0,1
140401,235792,NaN,https://www.leadcastingcall.com,30.0,www.leadcastingcall.com,23.0,NaN,NaN,1.000000,0.522907,...,0.0,0.0,1.0,25.0,NaN,NaN,87.0,1.0,93.0,1
140402,235794,NaN,https://www.fedarb.com,21.0,NaN,14.0,NaN,NaN,1.000000,0.522907,...,0.0,0.0,1.0,NaN,36.0,NaN,102.0,NaN,NaN,1


## Import Feature Specification from Dataset's Website

In [ ]:
import requests
import json

# https://archive.ics.uci.edu/dataset/967/phiusiil+phishing+url+dataset
URL = "https://archive.ics.uci.edu/api/trpc/donated_datasets.findById,keywords.findByDatasetId,variables.findTableByDatasetId,variableInfo.findTupleByDatasetId,edits.findByDatasetId,creators.findByDatasetId,descriptive.findTupleByDatasetId,evals.findByDatasetId,citation.findByDatasetId,files.findByDatasetId,files.findTableByDatasetId,notes.findByDatasetId,papers.findByDatasetId,donated_datasets.names,reviews.findByDatasetId,python.findRequestById,users.findById,citation.findCitationRequests?batch=1&input=%7B%220%22%3A%7B%22json%22%3A967%7D%2C%221%22%3A%7B%22json%22%3A967%7D%2C%222%22%3A%7B%22json%22%3A967%7D%2C%223%22%3A%7B%22json%22%3A967%7D%2C%224%22%3A%7B%22json%22%3A%7B%22datasetID%22%3A967%7D%7D%2C%225%22%3A%7B%22json%22%3A967%7D%2C%226%22%3A%7B%22json%22%3A967%7D%2C%227%22%3A%7B%22json%22%3A967%7D%2C%228%22%3A%7B%22json%22%3A967%7D%2C%229%22%3A%7B%22json%22%3A967%7D%2C%2210%22%3A%7B%22json%22%3A967%7D%2C%2211%22%3A%7B%22json%22%3A967%7D%2C%2212%22%3A%7B%22json%22%3A967%7D%2C%2213%22%3A%7B%22json%22%3Anull%2C%22meta%22%3A%7B%22values%22%3A%5B%22undefined%22%5D%7D%7D%2C%2214%22%3A%7B%22json%22%3A967%7D%2C%2215%22%3A%7B%22json%22%3A967%7D%2C%2216%22%3A%7B%22json%22%3A16997%7D%2C%2217%22%3A%7B%22json%22%3A967%7D%7D"

JSON = json.loads(requests.get(URL).text)
data_schema = JSON[0]["result"]["data"]["json"]["edits"][6]["data"]["variables"]

# 1. Split Training Set and Validation Set

Splitting the training and validation set works as an early diagnostic towards the performance of the model we train. This is done before the preprocessing steps to **avoid data leakage inbetween the sets**. If you want to use k-fold cross-validation, split the data later and do the cleaning and preprocessing separately for each split.

Note: For training, you should use the data contained in the `train` folder given by the TA. The `test` data is only used for kaggle submission.

In [ ]:
X = df.drop(['label', 'id', 'FILENAME'], axis=1)  # Features
y = df['label']  # Target variable

X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)
X_train_pipe_test = X_train.copy()
X_val_pipe_test = X_val.copy()

In [ ]:
X_train

,URL,URLLength,Domain,DomainLength,IsDomainIP,TLD,CharContinuationRate,TLDLegitimateProb,URLCharProb,TLDLength,...,Bank,Pay,Crypto,HasCopyrightInfo,NoOfImage,NoOfCSS,NoOfJS,NoOfSelfRef,NoOfEmptyRef,NoOfExternalRef
28021,https://www.yc-tp.com,NaN,NaN,13.0,0.0,com,0.600000,0.522907,NaN,NaN,...,0.0,0.0,NaN,NaN,30.0,12.0,NaN,NaN,0.0,138.0
71026,NaN,25.0,NaN,18.0,NaN,com,NaN,NaN,0.061236,3.0,...,NaN,0.0,NaN,1.0,88.0,11.0,16.0,211.0,0.0,NaN
89927,https://www.autodesk.co.uk,25.0,NaN,18.0,0.0,NaN,0.818182,0.028555,NaN,NaN,...,0.0,NaN,0.0,1.0,31.0,NaN,25.0,NaN,1.0,NaN
94654,https://www.marketmanila.com,27.0,www.marketmanila.com,20.0,0.0,NaN,1.000000,NaN,0.066401,3.0,...,NaN,1.0,0.0,NaN,NaN,26.0,23.0,105.0,0.0,NaN
35275,https://www.re-course.org,NaN,NaN,17.0,0.0,org,NaN,NaN,NaN,3.0,...,1.0,1.0,0.0,1.0,NaN,11.0,17.0,23.0,0.0,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
110268,https://www.gemathis.com,23.0,www.gemathis.com,16.0,0.0,com,1.000000,0.522907,NaN,3.0,...,0.0,NaN,0.0,1.0,NaN,2.0,NaN,106.0,7.0,NaN
119879,NaN,24.0,www.mural24.co.uk,NaN,0.0,uk,0.800000,NaN,NaN,2.0,...,0.0,1.0,0.0,NaN,166.0,3.0,7.0,NaN,1.0,NaN
103694,NaN,34.0,NaN,27.0,0.0,NaN,1.000000,NaN,NaN,3.0,...,1.0,1.0,0.0,NaN,NaN,NaN,NaN,42.0,NaN,25.0
131932,https://www.leinsterrugby.ie,27.0,NaN,20.0,0.0,ie,NaN,0.001588,0.052675,NaN,...,1.0,0.0,0.0,1.0,NaN,NaN,57.0,NaN,0.0,331.0


In [ ]:
X_val

,URL,URLLength,Domain,DomainLength,IsDomainIP,TLD,CharContinuationRate,TLDLegitimateProb,URLCharProb,TLDLength,...,Bank,Pay,Crypto,HasCopyrightInfo,NoOfImage,NoOfCSS,NoOfJS,NoOfSelfRef,NoOfEmptyRef,NoOfExternalRef
8409,https://www.citroencarclub.org.uk,NaN,NaN,25.0,0.0,uk,NaN,0.028555,0.061899,2.0,...,0.0,0.0,0.0,1.0,NaN,20.0,27.0,2.0,14.0,NaN
20959,https://www.flipyourtext.com,NaN,www.flipyourtext.com,NaN,0.0,com,NaN,0.522907,NaN,3.0,...,0.0,NaN,0.0,NaN,2.0,NaN,9.0,8.0,0.0,NaN
88797,NaN,25.0,NaN,18.0,0.0,NaN,NaN,0.522907,NaN,3.0,...,NaN,1.0,NaN,1.0,3.0,3.0,NaN,2.0,NaN,2.0
24345,https://www.elcalafate.gov.ar,28.0,NaN,NaN,NaN,NaN,NaN,NaN,0.062508,2.0,...,NaN,0.0,NaN,1.0,NaN,NaN,40.0,82.0,2.0,NaN
95019,https://www.dongwon.com,22.0,NaN,NaN,0.0,NaN,1.00,0.522907,0.062712,3.0,...,0.0,0.0,0.0,1.0,NaN,12.0,9.0,63.0,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
36268,https://www.rerumromanarum.com,29.0,www.rerumromanarum.com,22.0,0.0,NaN,1.00,NaN,0.067816,3.0,...,0.0,0.0,0.0,0.0,NaN,3.0,NaN,172.0,0.0,NaN
39295,https://www.ftlofaot.com,NaN,www.ftlofaot.com,16.0,0.0,NaN,1.00,0.522907,0.064400,3.0,...,NaN,1.0,NaN,NaN,NaN,NaN,NaN,424.0,NaN,NaN
111590,https://www.ancestry.com.au,NaN,www.ancestry.com.au,NaN,0.0,au,0.75,0.010086,0.067188,NaN,...,0.0,NaN,NaN,1.0,NaN,5.0,NaN,NaN,0.0,NaN
134414,https://www.winterthurer-zeitung.ch,34.0,NaN,27.0,0.0,ch,0.65,0.004983,0.053312,NaN,...,0.0,0.0,NaN,NaN,NaN,NaN,NaN,172.0,NaN,NaN


In [ ]:
# Daftar Kolom Categorical
categorical_columns = [
    'IsDomainIP', 'HasObfuscation', 'IsHTTPS', 'HasTitle', 'HasFavicon',
    'Robots', 'IsResponsive', 'HasDescription', 'HasSocialNet',
    'HasSubmitButton', 'HasHiddenFields', 'HasPasswordField', 'Bank',
    'Pay', 'Crypto', 'HasCopyrightInfo', 'Domain', 'URL',
    'TLD', 'Title', 'HasExternalFormSubmit','NoOfSelfRedirect', 'NoOfURLRedirect'
]

categorical_columns_filtered = [
    'IsDomainIP', 'HasObfuscation', 'IsHTTPS', 'HasTitle', 'HasFavicon',
    'Robots', 'IsResponsive', 'HasDescription', 'HasSocialNet',
    'HasSubmitButton', 'HasHiddenFields', 'HasPasswordField', 'Bank',
    'Pay', 'Crypto', 'HasCopyrightInfo','HasExternalFormSubmit', 'NoOfSelfRedirect', 'NoOfURLRedirect'
]

# DataFrame Categorical
X_train_categorical_columns = X_train[categorical_columns]
X_train_categorical_columns_filtered = X_train[categorical_columns_filtered]

In [ ]:
# Daftar Kolom Non-Categorical
numerical_columns = [col for col in X_train.columns if col not in categorical_columns]

# DataFrame dengan Hanya Kolom Non-Categorical
X_train_numerical_all = X_train[numerical_columns]

In [ ]:
continuous_columns = [
    'CharContinuationRate',
    'ObfuscationRatio',
    'DegitRatioInURL',
    'DomainTitleMatchScore',
    'URLTitleMatchScore',
    'TLDLegitimateProb',
    'URLCharProb',
    'LetterRatioInURL',
    'SpacialCharRatioInURL'
]
X_train_continuous = X_train[continuous_columns]

In [ ]:
discrete_columns = [i for i in numerical_columns if i not in continuous_columns]
X_train_discrete = X_train[discrete_columns]

# 2. Data Cleaning and Preprocessing

This step is the first thing to be done once a Data Scientist have grasped a general knowledge of the data. Raw data is **seldom ready for training**, therefore steps need to be taken to clean and format the data for the Machine Learning model to interpret.

By performing data cleaning and preprocessing, you ensure that your dataset is ready for model training, leading to more accurate and reliable machine learning results. These steps are essential for transforming raw data into a format that machine learning algorithms can effectively learn from and make predictions.

We will give some common methods for you to try, but you only have to **at least implement one method for each process**. For each step that you will do, **please explain the reason why did you do that process. Write it in a markdown cell under the code cell you wrote.**

## A. Data Cleaning

**Data cleaning** is the crucial first step in preparing your dataset for machine learning. Raw data collected from various sources is often messy and may contain errors, missing values, and inconsistencies. Data cleaning involves the following steps:

1. **Handling Missing Data:** Identify and address missing values in the dataset. This can include imputing missing values, removing rows or columns with excessive missing data, or using more advanced techniques like interpolation.

2. **Dealing with Outliers:** Identify and handle outliers, which are data points significantly different from the rest of the dataset. Outliers can be removed or transformed to improve model performance.

3. **Data Validation:** Check for data integrity and consistency. Ensure that data types are correct, categorical variables have consistent labels, and numerical values fall within expected ranges.

4. **Removing Duplicates:** Identify and remove duplicate rows, as they can skew the model's training process and evaluation metrics.

5. **Feature Engineering**: Create new features or modify existing ones to extract relevant information. This step can involve scaling, normalizing, or encoding features for better model interpretability.

### I. Handling Missing Data

Missing data can adversely affect the performance and accuracy of machine learning models. There are several strategies to handle missing data in machine learning:

1. **Data Imputation:**

    a. **Mean, Median, or Mode Imputation:** For numerical features, you can replace missing values with the mean, median, or mode of the non-missing values in the same feature. This method is simple and often effective when data is missing at random.

    b. **Constant Value Imputation:** You can replace missing values with a predefined constant value (e.g., 0) if it makes sense for your dataset and problem.

    c. **Imputation Using Predictive Models:** More advanced techniques involve using predictive models to estimate missing values. For example, you can train a regression model to predict missing numerical values or a classification model to predict missing categorical values.

2. **Deletion of Missing Data:**

    a. **Listwise Deletion:** In cases where the amount of missing data is relatively small, you can simply remove rows with missing values from your dataset. However, this approach can lead to a loss of valuable information.

    b. **Column (Feature) Deletion:** If a feature has a large number of missing values and is not critical for your analysis, you can consider removing that feature altogether.

3. **Domain-Specific Strategies:**

    a. **Domain Knowledge:** In some cases, domain knowledge can guide the imputation process. For example, if you know that missing values are related to a specific condition, you can impute them accordingly.

4. **Imputation Libraries:**

    a. **Scikit-Learn:** Scikit-Learn provides a `SimpleImputer` class that can handle basic imputation strategies like mean, median, and mode imputation.

    b. **Fancyimpute:** Fancyimpute is a Python library that offers more advanced imputation techniques, including matrix factorization, k-nearest neighbors, and deep learning-based methods.

The choice of imputation method should be guided by the nature of your data, the amount of missing data, the problem you are trying to solve, and the assumptions you are willing to make.

In [ ]:
X_train.isnull().sum()

### **X_Train**

#### **Kolom URLLength**

In [ ]:
def fill_url_length(data, url_col='URL', url_length_col='URLLength'):
    data[url_length_col] = data.apply(
        lambda row: len(str(row[url_col])) if pd.isnull(row[url_length_col]) and pd.notnull(row[url_col]) else row[url_length_col],
        axis=1
    )
    return data

X_train = fill_url_length(X_train, url_col='URL', url_length_col='URLLength')
X_train[['URL','URLLength']]

In [ ]:
X_train[X_train['URLLength'].isnull()]

#### **Kolom Domain**

In [ ]:
def fill_domain(data, url_col='URL', domain_col='Domain'):
    def get_domain(url):
        return urlparse(url).netloc if pd.notnull(url) else None

    data[domain_col] = data.apply(
        lambda row: get_domain(row[url_col]) if pd.isnull(row[domain_col]) and pd.notnull(row[url_col]) else row[domain_col],
        axis=1
    )
    return data

X_train = fill_domain(X_train, url_col='URL', domain_col='Domain')
X_train[['URL', 'Domain']]

In [ ]:
X_train[X_train['Domain'].isnull()]

#### **Kolom DomainLength**

In [ ]:
def fill_domain_length(data, domain_col='Domain', domain_length_col='DomainLength'):
    data[domain_length_col] = data.apply(
        lambda row: len(str(row[domain_col]))
        if pd.isnull(row[domain_length_col]) and pd.notnull(row[domain_col])
        else row[domain_length_col],
        axis=1
    )
    return data

X_train = fill_domain_length(X_train, domain_col='Domain', domain_length_col='DomainLength')
X_train[['Domain', 'DomainLength']]

In [ ]:
X_train[X_train['DomainLength'].isnull()]

#### **Kolom IsDomainIP**

In [ ]:
def fill_is_domain_ip(data, domain_col='Domain', is_domain_ip_col='IsDomainIP'):
    def is_ipaddress(domain):
        if pd.notnull(domain):
            ip_pattern = r'^(\d{1,3}\.){3}\d{1,3}$'
            if re.match(ip_pattern, domain):
                parts = domain.split('.')
                return all(0 <= int(part) <= 255 for part in parts)
        return False

    data[is_domain_ip_col] = data.apply(
        lambda row: (1 if is_ipaddress(row[domain_col]) else 0)
        if pd.isnull(row[is_domain_ip_col]) and pd.notnull(row[domain_col])
        else row[is_domain_ip_col],
        axis=1
    )
    return data

X_train = fill_is_domain_ip(X_train, domain_col='Domain', is_domain_ip_col='IsDomainIP')
X_train[['Domain', 'IsDomainIP']]

In [ ]:
X_train[X_train['IsDomainIP'].isnull()]

#### **Kolom TLD**

In [ ]:
def fill_tld(data, domain_col='Domain', tld_col='TLD'):
    def generate_tld(domain):
        if pd.notnull(domain):
            parts = domain.split('.')
            if len(parts) > 1:
                return parts[-1]
        return np.nan

    data[tld_col] = data.apply(
        lambda row: generate_tld(row[domain_col])
        if pd.isnull(row[tld_col])
        else row[tld_col],
        axis=1
    )
    return data

X_train = fill_tld(X_train, domain_col='Domain', tld_col='TLD')
X_train[['URL', 'Domain', 'TLD']]

In [ ]:
X_train[X_train['TLD'].isnull()]

#### **Kolom CharContinuationRate**

In [ ]:
def fill_char_continuation_rate(data, url_col='URL', char_rate_col='CharContinuationRate'):
    def generate_char_continuation_rate(url):
        if pd.notnull(url):
            sequences = re.findall(r'[a-zA-Z0-9_]+', url)
            total_sequence_length = sum(len(seq) for seq in sequences)
            total_url_length = len(url)
            return total_sequence_length / total_url_length if total_url_length > 0 else np.nan
        return np.nan

    data[char_rate_col] = data.apply(
        lambda row: generate_char_continuation_rate(row[url_col])
        if pd.isnull(row[char_rate_col]) else row[char_rate_col],
        axis=1
    )
    return data

X_train = fill_char_continuation_rate(X_train, url_col='URL', char_rate_col='CharContinuationRate')
X_train[['URL', 'CharContinuationRate']]

In [ ]:
X_train[X_train['CharContinuationRate'].isnull()]

#### **Kolom TLDLegitimateProb**

In [ ]:
# Distribution analysis to determine the imputation method
def fill_tld_legitimate_prob(data, tld_col='TLD', tld_prob_col='TLDLegitimateProb', is_domain_ip_col='IsDomainIP'):

    skewness = data[tld_prob_col].skew()
    if skewness > 1 or skewness < -1:
        global_fill_value = data[tld_prob_col].median()
        print("Uses median for imputation because distribution is skewed.")
    else:
        global_fill_value = data[tld_prob_col].mean()
        print("Uses the mean for imputation because of the normal distribution.")

    print(f"Global fill value: {global_fill_value}")


    tld_prob_mean = data.groupby(tld_col)[tld_prob_col].mean()


    def fill_tld_legit_prob(row):
        if pd.isnull(row[tld_prob_col]):
            if row[is_domain_ip_col] == 1:
                return 0
            if pd.notnull(row[tld_col]) and row[tld_col] in tld_prob_mean.index:  # TLD not found
                return tld_prob_mean[row[tld_col]]
            if pd.isnull(row[tld_col]) and pd.isnull(row[is_domain_ip_col]):  # TLD dan IsDomainIP not found
                return global_fill_value
        return row[tld_prob_col]
    data[tld_prob_col] = data.apply(fill_tld_legit_prob, axis=1)
    data[tld_prob_col] = data[tld_prob_col].fillna(global_fill_value)

    return data

X_train = fill_tld_legitimate_prob(
    X_train,
    tld_col='TLD',
    tld_prob_col='TLDLegitimateProb',
    is_domain_ip_col='IsDomainIP'
)
X_train[['TLD', 'TLDLegitimateProb']]

In [ ]:
X_train[X_train['TLDLegitimateProb'].isnull()]

#### **Kolom URLCharProb**

In [ ]:
# Function to calculate character probabilities
def calculate_char_prob(df, url_col):
    char_count = {}
    total_chars = 0
    for url in df[url_col].dropna():
        for char in url.lower():
            if char.isalnum():  # Only consider alphanumeric characters
                char_count[char] = char_count.get(char, 0) + 1
                total_chars += 1
    return {char: count / total_chars for char, count in char_count.items()}

# Function to calculate URLCharProb
def fill_url_char_prob(data, url_col='URL', char_prob_col='URLCharProb', char_prob=None):
    if char_prob is None:
        raise ValueError("Character probabilities (`char_prob`) must be provided.")

    def calculate_url_char_prob(url):
        if pd.notnull(url):
            total_prob = sum(char_prob.get(char, 0) for char in url.lower() if char.isalnum())
            n = len(url)
            return total_prob / n if n > 0 else np.nan
        return np.nan

    data[char_prob_col] = data.apply(
        lambda row: calculate_url_char_prob(row[url_col])
        if pd.isnull(row[char_prob_col]) else row[char_prob_col],
        axis=1
    )
    return data


char_prob = calculate_char_prob(X_train, url_col='URL')
X_train = fill_url_char_prob(X_train, url_col='URL', char_prob_col='URLCharProb', char_prob=char_prob)
X_train[['URL', 'URLCharProb']]

In [ ]:
X_train[X_train['URLCharProb'].isnull()]

#### **Kolom TLDLength**

In [ ]:
def fill_tld_length(data, tld_col='TLD', tld_length_col='TLDLength'):
    def calculate_tld_length(tld):
        if pd.notnull(tld):
            return len(str(tld))
        return np.nan

    data[tld_length_col] = data.apply(
        lambda row: calculate_tld_length(row[tld_col])
        if pd.isnull(row[tld_length_col]) else row[tld_length_col],
        axis=1
    )
    return data

X_train = fill_tld_length(X_train, tld_col='TLD', tld_length_col='TLDLength')
X_train[['TLD', 'TLDLength']]

In [ ]:
X_train[X_train['TLDLength'].isnull()]

#### **Kolom NoOfSubDomain**

In [ ]:
def fill_no_of_subdomains(data, domain_col='Domain', subdomain_col='NoOfSubDomain'):
    def calculate_no_of_subdomains(domain):
        if pd.notnull(domain):
            parts = domain.split('.')
            return len(parts) - 2 if len(parts) > 2 else 0
        return np.nan

    data[subdomain_col] = data.apply(
        lambda row: calculate_no_of_subdomains(row[domain_col])
        if pd.isnull(row[subdomain_col]) else row[subdomain_col],
        axis=1
    )
    return data

X_train = fill_no_of_subdomains(X_train, domain_col='Domain', subdomain_col='NoOfSubDomain')
X_train[['Domain', 'NoOfSubDomain']]

In [ ]:
X_train[X_train['NoOfSubDomain'].isnull()]

#### **Kolom HasObfuscation**

In [ ]:
def fill_has_obfuscation(data, url_col='URL', obfuscation_col='HasObfuscation'):
    def detect_advanced_obfuscation(url):
        if pd.notnull(url):
            if len(re.findall(r'[-_]', url)) > 3:  # Rule 1: Too many special characters
                return 1
            if re.search(r'[a-zA-Z]+\d+|\d+[a-zA-Z]+', url):  # Rule 2: Mixed alphanumeric patterns
                return 1
            if len(url) % 4 == 0 and re.match(r'^[A-Za-z0-9+/]*={0,2}$', url):  # Rule 3: Base64 encoded strings
                try:
                    base64.b64decode(url, validate=True)
                    return 1
                except Exception:
                    pass
            reversed_url = url[::-1]  # Rule 4: Reversed strings
            if re.match(r'^[a-zA-Z0-9.\-]+$', reversed_url):
                return 1
            if not re.search(r'[a-zA-Z]{3,}', url):  # Rule 5: Randomized strings without meaningful words
                return 1
            return 0  # No obfuscation detected
        return 0  # Missing values treated as no obfuscation

    data[obfuscation_col] = data.apply(
        lambda row: detect_advanced_obfuscation(row[url_col])
        if pd.isnull(row[obfuscation_col]) and pd.notnull(row[url_col])
        else row[obfuscation_col],
        axis=1
    )
    data[obfuscation_col] = data[obfuscation_col].apply(lambda x: 1 if x == True else 0)
    return data

X_train = fill_has_obfuscation(X_train, url_col='URL', obfuscation_col='HasObfuscation')
X_train[['URL', 'HasObfuscation']]

In [ ]:
X_train[X_train['HasObfuscation'].isnull()]

#### **Kolom NoOfObfuscatedChar**

In [ ]:
def fill_no_of_obfuscated_characters(data, url_col='URL', obf_char_col='NoOfObfuscatedChar'):
    def count_obfuscated_characters(url):
        if pd.notnull(url):
            hex_count = len(re.findall(r'%[0-9a-fA-F]{2}', url))  # Count %XX hexadecimal patterns
            at_count = url.count('@')  # Count @ symbol
            return hex_count + at_count  # Total obfuscated characters
        return np.nan

    data[obf_char_col] = data.apply(
        lambda row: count_obfuscated_characters(row[url_col])
        if pd.isnull(row[obf_char_col]) else row[obf_char_col],
        axis=1
    )
    return data

X_train = fill_no_of_obfuscated_characters(X_train, url_col='URL', obf_char_col='NoOfObfuscatedChar')
X_train[['URL', 'NoOfObfuscatedChar']]

In [ ]:
X_train[X_train['NoOfObfuscatedChar'].isnull()]

#### **Kolom ObfuscationRatio**

In [ ]:
def fill_obfuscation_ratio(data, obf_char_col='NoOfObfuscatedChar', url_length_col='URLLength', obf_ratio_col='ObfuscationRatio'):
    def calculate_obfuscation_ratio(no_of_obfchar, url_length):
        if pd.notnull(no_of_obfchar) and pd.notnull(url_length) and url_length > 0:
            return no_of_obfchar / url_length
        return np.nan

    data[obf_ratio_col] = data.apply(
        lambda row: calculate_obfuscation_ratio(row[obf_char_col], row[url_length_col])
        if pd.isnull(row[obf_ratio_col]) else row[obf_ratio_col],
        axis=1
    )
    return data

X_train = fill_obfuscation_ratio(X_train, obf_char_col='NoOfObfuscatedChar', url_length_col='URLLength', obf_ratio_col='ObfuscationRatio')
X_train[['NoOfObfuscatedChar', 'URLLength', 'ObfuscationRatio']]

In [ ]:
X_train[X_train['ObfuscationRatio'].isnull()]

#### **Kolom NoOfLettersInURL**  

In [ ]:
def fill_no_of_letters_in_url(data, url_col='URL', letters_col='NoOfLettersInURL'):
    def calculate_no_of_letters(url):
        if pd.notnull(url):
            return sum(c.isalpha() for c in url)
        return np.nan

    data[letters_col] = data.apply(
        lambda row: calculate_no_of_letters(row[url_col])
        if pd.isnull(row[letters_col]) else row[letters_col],
        axis=1
    )
    return data

X_train = fill_no_of_letters_in_url(X_train, url_col='URL', letters_col='NoOfLettersInURL')
X_train[['URL', 'NoOfLettersInURL']]

In [ ]:
X_train[X_train['NoOfLettersInURL'].isnull()]

#### **Kolom LetterRatioInURL**

In [ ]:
def fill_letter_ratio_in_url(data, letters_col='NoOfLettersInURL', url_length_col='URLLength', ratio_col='LetterRatioInURL'):
    def calculate_letter_ratio(no_of_letters, url_length):
        if pd.notnull(no_of_letters) and pd.notnull(url_length) and url_length > 0:
            return no_of_letters / url_length
        return np.nan

    data[ratio_col] = data.apply(
        lambda row: calculate_letter_ratio(row[letters_col], row[url_length_col])
        if pd.isnull(row[ratio_col]) else row[ratio_col],
        axis=1
    )
    return data

X_train = fill_letter_ratio_in_url(X_train, letters_col='NoOfLettersInURL', url_length_col='URLLength', ratio_col='LetterRatioInURL')
X_train[['NoOfLettersInURL', 'URLLength', 'LetterRatioInURL']]

In [ ]:
X_train[X_train['LetterRatioInURL'].isnull()]

#### **Kolom NoOfDigitsInURL**

In [ ]:
def fill_no_of_digits_in_url(data, url_col='URL', digits_col='NoOfDegitsInURL'):
    def calculate_no_of_digits(url):
        if pd.notnull(url):
            return sum(c.isdigit() for c in url)
        return np.nan

    data[digits_col] = data.apply(
        lambda row: calculate_no_of_digits(row[url_col])
        if pd.isnull(row[digits_col]) else row[digits_col],
        axis=1

    )
    return data

X_train = fill_no_of_digits_in_url(X_train, url_col='URL', digits_col='NoOfDegitsInURL')
X_train[['URL', 'NoOfDegitsInURL']]

In [ ]:
X_train[X_train['NoOfDegitsInURL'].isnull()]

#### **Kolom DegitRatioInURL**

In [ ]:
def fill_digits_ratio_in_url(data, url_col='URL', ratio_col='DegitRatioInURL'):
    def calculate_digits_ratio(url):
        if pd.notnull(url) and len(url) > 0:
            return sum(c.isdigit() for c in url) / len(url)
        return np.nan

    data[ratio_col] = data.apply(
        lambda row: calculate_digits_ratio(row[url_col])
        if pd.isnull(row[ratio_col]) else row[ratio_col],
        axis=1
    )
    return data

X_train = fill_digits_ratio_in_url(X_train, url_col='URL', ratio_col='DegitRatioInURL')
X_train[['URL', 'DegitRatioInURL']]

In [ ]:
X_train[X_train['DegitRatioInURL'].isnull()]

#### **Kolom NoOfEqualsInURL**

In [ ]:
def fill_no_of_equals_in_url(data, url_col='URL', equals_col='NoOfEqualsInURL'):
    def calculate_no_of_equals(url):
        if pd.notnull(url):
            return sum(c == "=" for c in url)
        return np.nan

    data[equals_col] = data.apply(
        lambda row: calculate_no_of_equals(row[url_col])
        if pd.isnull(row[equals_col]) else row[equals_col],
        axis=1
    )
    return data

X_train = fill_no_of_equals_in_url(X_train, url_col='URL', equals_col='NoOfEqualsInURL')
X_train[['URL', 'NoOfEqualsInURL']]

In [ ]:
X_train[X_train['NoOfEqualsInURL'].isnull()]

#### **Kolom NoOfQMarkInURL**

In [ ]:
def fill_no_of_qmark_in_url(data, url_col='URL', qmark_col='NoOfQMarkInURL'):
    def calculate_no_of_qmark(url):
        if pd.notnull(url):
            return sum(c == "?" for c in url)
        return np.nan

    data[qmark_col] = data.apply(
        lambda row: calculate_no_of_qmark(row[url_col])
        if pd.isnull(row[qmark_col]) else row[qmark_col],
        axis=1
    )
    return data

X_train = fill_no_of_qmark_in_url(X_train, url_col='URL', qmark_col='NoOfQMarkInURL')
X_train[['URL', 'NoOfQMarkInURL']]

In [ ]:
X_train[X_train['NoOfQMarkInURL'].isnull()]

#### **Kolom NoOfAmpersandInURL**

In [ ]:
def fill_no_of_ampersand_in_url(data, url_col='URL', ampersand_col='NoOfAmpersandInURL'):
    def calculate_no_of_ampersand(url):
        if pd.notnull(url):
            return sum(c == "&" for c in url)
        return np.nan

    data[ampersand_col] = data.apply(
        lambda row: calculate_no_of_ampersand(row[url_col])
        if pd.isnull(row[ampersand_col]) else row[ampersand_col],
        axis=1
    )
    return data

X_train = fill_no_of_ampersand_in_url(X_train, url_col='URL', ampersand_col='NoOfAmpersandInURL')
X_train[['URL', 'NoOfAmpersandInURL']]

In [ ]:
X_train[X_train['NoOfAmpersandInURL'].isnull()]

#### **Kolom NoOfOtherSpecialCharsInURL**

In [ ]:
def fill_no_of_special_chars_in_url(data, url_col='URL', special_chars_col='NoOfOtherSpecialCharsInURL'):
    def calculate_no_of_specials(url):
        if pd.notnull(url):
            return sum(c.lower() not in "0123456789abcdefghijklmnopqrstuvwxyz" for c in url)
        return np.nan

    data[special_chars_col] = data.apply(
        lambda row: calculate_no_of_specials(row[url_col])
        if pd.isnull(row[special_chars_col]) else row[special_chars_col],
        axis=1
    )
    return data

X_train = fill_no_of_special_chars_in_url(X_train, url_col='URL', special_chars_col='NoOfOtherSpecialCharsInURL')
X_train[['URL', 'NoOfOtherSpecialCharsInURL']]

In [ ]:
X_train[X_train['NoOfOtherSpecialCharsInURL'].isnull()]

#### **Kolom SpacialCharRatioInURL**

In [ ]:
def fill_specials_ratio_in_url(data, url_col='URL', ratio_col='SpacialCharRatioInURL'):
    def calculate_specials_ratio(url):
        if pd.notnull(url) and len(url) > 0:
            return sum(c.lower() not in "0123456789abcdefghijklmnopqrstuvwxyz" for c in url) / len(url)
        return np.nan

    data[ratio_col] = data.apply(
        lambda row: calculate_specials_ratio(row[url_col])
        if pd.isnull(row[ratio_col]) else row[ratio_col],
        axis=1
    )
    return data

X_train = fill_specials_ratio_in_url(X_train, url_col='URL', ratio_col='SpacialCharRatioInURL')
X_train[['URL', 'SpacialCharRatioInURL']]

In [ ]:
X_train[X_train['SpacialCharRatioInURL'].isnull()]

#### **Kolom IsHTTPS**

In [ ]:
def fill_is_https(data, url_col='URL', https_col='IsHTTPS'):
    def calculate_ishttps(url):
        if pd.notnull(url):
            return 1 if "https://" in url else 0
        return np.nan

    data[https_col] = data.apply(
        lambda row: calculate_ishttps(row[url_col])
        if pd.isnull(row[https_col]) else row[https_col],
        axis=1
    )
    return data

X_train = fill_is_https(X_train, url_col='URL', https_col='IsHTTPS')
X_train[['URL', 'IsHTTPS']]

In [ ]:
X_train[X_train['IsHTTPS'].isnull()]

#### **Kolom HasTitle**

In [ ]:
def fill_has_title(data, title_col='Title', has_title_col='HasTitle'):
    data[has_title_col] = data.apply(
        lambda row: row[has_title_col]
        if not pd.isnull(row[has_title_col])
        else (1 if not pd.isnull(row[title_col]) else row[has_title_col]),
        axis=1
    )
    return data

X_train = fill_has_title(X_train, title_col='Title', has_title_col='HasTitle')
X_train[['Title', 'HasTitle']]

In [ ]:
X_train[X_train['HasTitle'].isnull()]

#### **Kolom DomainTitleMatchScore**

In [ ]:
def fill_domain_title_match_score(data, title_col='Title', domain_col='Domain', score_col='DomainTitleMatchScore'):
    def calculate_match_score(title, domain):
        tSet = title.split(" ") if pd.notnull(title) else []
        txtDomain = domain.split(".")[:-1] if pd.notnull(domain) else []
        txtDomain = [i for i in txtDomain if i != "www"]
        txtDomain = ".".join(txtDomain)

        score = 0
        baseScore = 100 / len(txtDomain) if len(txtDomain) > 0 else 0

        for element in tSet:
            if element in txtDomain:
                n = len(element)
                score += baseScore * n
                txtDomain = txtDomain.replace(element, "")
                if score > 99.9:
                    score = 100
        return score

    data[score_col] = data.apply(
        lambda row: calculate_match_score(row[title_col], row[domain_col])
        if pd.isnull(row[score_col])
        and pd.notnull(row[title_col])
        and pd.notnull(row[domain_col])
        else row[score_col],
        axis=1
    )
    return data

X_train = fill_domain_title_match_score(
    X_train,
    title_col='Title',
    domain_col='Domain',
    score_col='DomainTitleMatchScore'
)
X_train[['URL', 'Title', 'DomainTitleMatchScore']]

In [ ]:
X_train[X_train['DomainTitleMatchScore'].isnull()]

#### **Kolom URLTitleMatchScore**

In [ ]:
def fill_url_title_match_score(data, title_col='Title', url_col='URL', score_col='URLTitleMatchScore'):
    def calculate_url_title_match_score(title, url):
        if pd.notnull(title) and pd.notnull(url):
            tSet = title.split(" ")
            txtURL = (
                urlparse(url).netloc.split(".")[:-1] +
                [i for i in urlparse(url).path.split("/") if i != ""]
            )
            txtURL = [i for i in txtURL if i != "www"]
            txtURL = ".".join(txtURL)

            score = 0
            baseScore = 100 / len(txtURL) if len(txtURL) > 0 else 0

            for element in tSet:
                if element in txtURL:
                    n = len(element)
                    score += baseScore * n
                    txtURL = txtURL.replace(element, "")
                    if score > 99.9:
                        score = 100
            return score
        return np.nan

    data[score_col] = data.apply(
        lambda row: calculate_url_title_match_score(row[title_col], row[url_col])
        if pd.isnull(row[score_col]) else row[score_col],
        axis=1
    )
    return data

X_train = fill_url_title_match_score(
    X_train,
    title_col='Title',
    url_col='URL',
    score_col='URLTitleMatchScore'
)
X_train[['URL', 'Title', 'URLTitleMatchScore']]

In [ ]:
X_train[X_train['URLTitleMatchScore'].isnull()]

In [ ]:
X_train.isnull().sum().sort_values()

In [ ]:
np.median(X_train.isnull().sum())

#### Handling Missing Value by Data Imputation

In [ ]:
def fill_numerical_columns(data, numerical_columns, skewness_threshold=3):
    for col in numerical_columns:
        if col in data.columns:
            skewness = data[col].skew()
            if skewness > skewness_threshold or skewness < -skewness_threshold:
                fill_value = data[col].median()
                print(f"Column {col}: Using median for imputation because distribution is highly skewed.")
            else:
                fill_value = data[col].mean()
                print(f"Column {col}: Using mean for imputation because distribution is approximately normal.")
            data[col] = data[col].fillna(fill_value)
    print("Missing values after imputation:")
    print(data[numerical_columns].isnull().sum())
    return data
X_train = fill_numerical_columns(X_train, numerical_columns, skewness_threshold=3)

In [ ]:
X_train.isnull().sum().sort_values()

In [ ]:
def fill_categorical_columns(data, categorical_columns, initial_group_by):
    group_by_columns = initial_group_by.copy()

    for col in categorical_columns:
        mode_values = (
            data.groupby(group_by_columns)[col]
            .agg(lambda x: x.mode()[0] if not x.mode().empty else None)
        )

        global_mode = data[col].mode()[0] if not data[col].mode().empty else None

        def fill_value(row):
            if pd.isnull(row[col]):
                group_key = tuple(row[gb] for gb in group_by_columns)
                return mode_values.get(group_key, global_mode)
            return row[col]

        data[col] = data.apply(fill_value, axis=1)

        group_by_columns.append(col)

    for col in categorical_columns:
        if data[col].isnull().sum() > 0:
            global_mode = data[col].mode()[0] if not data[col].mode().empty else None
            data[col] = data[col].fillna(global_mode)

    return data

categorical_columns_to_fill = [
    'IsDomainIP', 'IsHTTPS', 'HasTitle', 'IsResponsive', 'Pay', 'HasHiddenFields', 'Robots',
    'Crypto', 'HasDescription', 'Bank', 'HasExternalFormSubmit', 'HasFavicon',
    'HasSubmitButton', 'HasPasswordField', 'NoOfSelfRedirect', 'HasCopyrightInfo',
    'NoOfURLRedirect', 'HasSocialNet'
]

group_by_columns = ['HasObfuscation']

X_train = fill_categorical_columns(X_train, categorical_columns_to_fill, group_by_columns)
print(f"Missing values after imputation:")
print(X_train.isnull().sum())

In [ ]:
X_train.isnull().sum().sort_values()

### **X_val**

#### **Kolom URLLength**

In [ ]:
X_val = fill_url_length(X_val, url_col='URL', url_length_col='URLLength')
X_val[['URL','URLLength']]

#### **Kolom Domain**

In [ ]:
X_val = fill_domain(X_val, url_col='URL', domain_col='Domain')
X_val[['URL', 'Domain']]

#### **Kolom DomainLength**

In [ ]:
X_val = fill_domain_length(X_val, domain_col='Domain', domain_length_col='DomainLength')
X_val[['Domain', 'DomainLength']]

#### **Kolom IsDomainIP**

In [ ]:
X_val = fill_is_domain_ip(X_val, domain_col='Domain', is_domain_ip_col='IsDomainIP')
X_val[['Domain', 'IsDomainIP']]

#### **Kolom TLD**

In [ ]:
X_val = fill_tld(X_val, domain_col='Domain', tld_col='TLD')
X_val[['URL', 'Domain', 'TLD']]

#### **Kolom CharContinuationRate**

In [ ]:
X_val = fill_char_continuation_rate(X_val, url_col='URL', char_rate_col='CharContinuationRate')
X_val[['URL', 'CharContinuationRate']]

#### **Kolom TLDLegitimateProb**

In [ ]:
X_val = fill_tld_legitimate_prob(
    X_val,
    tld_col='TLD',
    tld_prob_col='TLDLegitimateProb',
    is_domain_ip_col='IsDomainIP'
)
X_val[['TLD', 'TLDLegitimateProb']]

#### **Kolom URLCharProb**

In [ ]:
char_prob = calculate_char_prob(X_val, url_col='URL')
X_val = fill_url_char_prob(X_val, url_col='URL', char_prob_col='URLCharProb', char_prob=char_prob)
X_val[['URL', 'URLCharProb']]

#### **Kolom TLDLength**

In [ ]:
X_val = fill_tld_length(X_val, tld_col='TLD', tld_length_col='TLDLength')
X_val[['TLD', 'TLDLength']]

#### **Kolom NoOfSubDomain**

In [ ]:
X_val = fill_no_of_subdomains(X_val, domain_col='Domain', subdomain_col='NoOfSubDomain')
X_val[['Domain', 'NoOfSubDomain']]

#### **Kolom HasObfuscation**

In [ ]:
X_val = fill_has_obfuscation(X_val, url_col='URL', obfuscation_col='HasObfuscation')
X_val[['URL', 'HasObfuscation']]

#### **Kolom NoOfObfuscatedChar**

In [ ]:
X_val = fill_no_of_obfuscated_characters(X_val, url_col='URL', obf_char_col='NoOfObfuscatedChar')
X_val[['URL', 'NoOfObfuscatedChar']]

#### **Kolom ObfuscationRatio**

In [ ]:
X_val = fill_obfuscation_ratio(X_val, obf_char_col='NoOfObfuscatedChar', url_length_col='URLLength', obf_ratio_col='ObfuscationRatio')
X_val[['NoOfObfuscatedChar', 'URLLength', 'ObfuscationRatio']]

#### **Kolom NoOfLettersInURL**  

In [ ]:
X_val = fill_no_of_letters_in_url(X_val, url_col='URL', letters_col='NoOfLettersInURL')
X_val[['URL', 'NoOfLettersInURL']]

#### **Kolom LetterRatioInURL**

In [ ]:
X_val = fill_letter_ratio_in_url(X_val, letters_col='NoOfLettersInURL', url_length_col='URLLength', ratio_col='LetterRatioInURL')
X_val[['NoOfLettersInURL', 'URLLength', 'LetterRatioInURL']]

#### **Kolom NoOfDigitsInURL**

In [ ]:
X_val = fill_no_of_digits_in_url(X_val, url_col='URL', digits_col='NoOfDegitsInURL')
X_val[['URL', 'NoOfDegitsInURL']]

#### **Kolom DegitRatioInURL**

In [ ]:
X_val = fill_digits_ratio_in_url(X_val, url_col='URL', ratio_col='DegitRatioInURL')
X_val[['URL', 'DegitRatioInURL']]

#### **Kolom NoOfEqualsInURL**

In [ ]:
X_val = fill_no_of_equals_in_url(X_val, url_col='URL', equals_col='NoOfEqualsInURL')
X_val[['URL', 'NoOfEqualsInURL']]

#### **Kolom NoOfQMarkInURL**

In [ ]:
X_val = fill_no_of_qmark_in_url(X_val, url_col='URL', qmark_col='NoOfQMarkInURL')
X_val[['URL', 'NoOfQMarkInURL']]

#### **Kolom NoOfAmpersandInURL**

In [ ]:
X_val = fill_no_of_ampersand_in_url(X_val, url_col='URL', ampersand_col='NoOfAmpersandInURL')
X_val[['URL', 'NoOfAmpersandInURL']]

#### **Kolom NoOfOtherSpecialCharsInURL**

In [ ]:
X_val = fill_no_of_special_chars_in_url(X_val, url_col='URL', special_chars_col='NoOfOtherSpecialCharsInURL')
X_val[['URL', 'NoOfOtherSpecialCharsInURL']]

#### **Kolom SpacialCharRatioInURL**

In [ ]:
X_val = fill_specials_ratio_in_url(X_val, url_col='URL', ratio_col='SpacialCharRatioInURL')
X_val[['URL', 'SpacialCharRatioInURL']]

#### **Kolom IsHTTPS**

In [ ]:
X_val = fill_is_https(X_val, url_col='URL', https_col='IsHTTPS')
X_val[['URL', 'IsHTTPS']]

#### **Kolom HasTitle**

In [ ]:
X_val = fill_has_title(X_val, title_col='Title', has_title_col='HasTitle')
X_val[['Title', 'HasTitle']]

#### **Kolom DomainTitleMatchScore**

In [ ]:
X_val = fill_domain_title_match_score(
    X_val,
    title_col='Title',
    domain_col='Domain',
    score_col='DomainTitleMatchScore'
)
X_val[['URL', 'Title', 'DomainTitleMatchScore']]

#### **Kolom URLTitleMatchScore**

In [ ]:
X_val = fill_url_title_match_score(
    X_val,
    title_col='Title',
    url_col='URL',
    score_col='URLTitleMatchScore'
)
X_val[['URL', 'Title', 'URLTitleMatchScore']]

In [ ]:
X_val.isnull().sum().sort_values()

#### Handling Missing Value by Data Imputation

In [ ]:
X_val = fill_numerical_columns(X_val, numerical_columns, skewness_threshold=3)

In [ ]:
X_val.isnull().sum().sort_values()

In [ ]:
X_val = fill_categorical_columns(X_val, categorical_columns_to_fill, group_by_columns)
print(f"Missing values after imputation:")
print(X_val.isnull().sum())

In [ ]:
X_val.isnull().sum().sort_values()

#### **Update Grouping DF**

In [ ]:
X_train_categorical_columns = X_train[categorical_columns]
X_train_categorical_columns_filtered = X_train[categorical_columns_filtered]
X_train_numerical_all = X_train[numerical_columns]
X_train_continuous = X_train[continuous_columns]
X_train_discrete = X_train[discrete_columns]

X_val_categorical_columns = X_val[categorical_columns]
X_val_categorical_columns_filtered = X_val[categorical_columns_filtered]
X_val_numerical_all = X_val[numerical_columns]
X_val_continuous = X_val[continuous_columns]
X_val_discrete = X_val[discrete_columns]

### II. Dealing with Outliers

Outliers are data points that significantly differ from the majority of the data. They can be unusually high or low values that do not fit the pattern of the rest of the dataset. Outliers can significantly impact model performance, so it is important to handle them properly.

Some methods to handle outliers:
1. **Imputation**: Replace with mean, median, or a boundary value.
2. **Clipping**: Cap values to upper and lower limits.
3. **Transformation**: Use log, square root, or power transformations to reduce their influence.
4. **Model-Based**: Use algorithms robust to outliers (e.g., tree-based models, Huber regression).

#### Outliers For X_train

##### Mendeteksi outliers before handling

In [ ]:
# Cek box plot for all numerical data

cols_per_row = 4
num_cols = X_train_numerical_all.shape[1]
num_rows = (num_cols + cols_per_row - 1) // cols_per_row

fig, axes = plt.subplots(nrows=num_rows, ncols=cols_per_row, figsize=(16, 4 * num_rows))
axes = axes.flatten()

for i, column in enumerate(X_train_numerical_all.columns):
    X_train_numerical_all.boxplot(column=column, ax=axes[i])
    axes[i].set_title(f'Box Plot for {column}')
    axes[i].set_ylabel('Values')

for j in range(i + 1, len(axes)):
    axes[j].set_visible(False)

plt.tight_layout()
plt.show()

In [ ]:
# Function to count outliers

def detect_outliers(df, skewness_threshold=3, z_score_threshold=3):
    results = []

    for col in df.columns:

        skewness = df[col].skew()

        if abs(skewness) <= skewness_threshold:
            z_scores = zscore(df[col].dropna())
            outliers = np.abs(z_scores) > z_score_threshold
            method = "Z-Score"
        else:

            Q1 = df[col].quantile(0.25)
            Q3 = df[col].quantile(0.75)
            IQR = Q3 - Q1
            lower_bound = Q1 - 1.5 * IQR
            upper_bound = Q3 + 1.5 * IQR
            outliers = (df[col] < lower_bound) | (df[col] > upper_bound)
            method = "IQR"

        outlier_count = outliers.sum()

        results.append({
            'Feature': col,
            'Outlier Count': outlier_count,
            'Method': method
        })

    return pd.DataFrame(results)

In [ ]:
# Count Outliers before Handling

outlier_summary = detect_outliers(X_train_numerical_all)

print(outlier_summary)

#### Handling Outliers

##### Logform for data in a range 0-1000

In [ ]:
def outlier_handlers_logform(X):
  X_numerical_filtered = X_train[numerical_columns].loc[:, (X_train[numerical_columns].max() <= 1000) & (X_train[numerical_columns].min() >= 0)]
  return np.log1p(X_numerical_filtered)

# X_train_numerical_filtered = X_train_numerical_all.loc[:, (X_train_numerical_all.max() <= 1000) & (X_train_numerical_all.min() >= 0)]

# X_train_log_transformed_filtered = np.log1p(X_train_numerical_filtered)

X_train_log_transformed_filtered = outlier_handlers_logform(X_train)
cols_per_row = 4
num_cols = X_train_log_transformed_filtered.shape[1]
num_rows = (num_cols + cols_per_row - 1) // cols_per_row

fig, axes = plt.subplots(nrows=num_rows, ncols=cols_per_row, figsize=(16, 4 * num_rows))
axes = axes.flatten()

for i, column in enumerate(X_train_log_transformed_filtered.columns):
    X_train_log_transformed_filtered.boxplot(column=column, ax=axes[i])
    axes[i].set_title(f'Log Transformed Box Plot for {column}')
    axes[i].set_ylabel('Log Values')

for j in range(i + 1, len(axes)):
    axes[j].set_visible(False)

plt.tight_layout()
plt.show()

##### SQRT untuk Data dengan rentang terdekat

In [ ]:
def outlier_handlers_sqrt(X):
  X_small_range = X_train[numerical_columns].loc[:, (X_train[numerical_columns].max() <= 15) & (X_train[numerical_columns].min() >= 0)]
  return np.sqrt(X_small_range)


X_train_sqrt_transformed = outlier_handlers_sqrt(X_train)

cols_per_row = 4
num_cols = X_train_sqrt_transformed.shape[1]
num_rows = (num_cols + cols_per_row - 1) // cols_per_row

fig, axes = plt.subplots(nrows=num_rows, ncols=cols_per_row, figsize=(16, 4 * num_rows))
axes = axes.flatten()

for i, column in enumerate(X_train_sqrt_transformed.columns):
    X_train_sqrt_transformed.boxplot(column=column, ax=axes[i])
    axes[i].set_title(f'Sqrt Transformed Box Plot for {column}')
    axes[i].set_ylabel('Sqrt Values')

for j in range(i + 1, len(axes)):
    axes[j].set_visible(False)

plt.tight_layout()
plt.show()

##### Clip

In [ ]:
def outlier_handlers_clip(df, lower_percentile=0.01, upper_percentile=0.99):
    clipped_df = df.copy()
    for col in clipped_df[numerical_columns].columns:
        lower_bound = np.percentile(clipped_df[col].dropna(), lower_percentile * 100)
        upper_bound = np.percentile(clipped_df[col].dropna(), upper_percentile * 100)
        clipped_df[col] = np.clip(clipped_df[col], lower_bound, upper_bound)

    return clipped_df

In [ ]:
X_train_numerical_all_clipped = outlier_handlers_clip(X_train_numerical_all)
print("\nClipped Data Head:")
print(X_train_numerical_all_clipped.head())

####  Mendeteksi outliers after handling

##### After Logtransform

In [ ]:
log_outlier_results = detect_outliers(X_train_log_transformed_filtered)
print("Log-Transformed Data Outliers:")
print(log_outlier_results)

##### After SQRT

In [ ]:
# Check outliers in sqrt-transformed data
sqrt_outlier_results = detect_outliers(X_train_sqrt_transformed)
print("\nSqrt-Transformed Data Outliers:")
print(sqrt_outlier_results)

##### After Clip

In [ ]:
# Check outliers in sqrt-transformed data
clip_outlier_results = detect_outliers(X_train_numerical_all_clipped)
print("\nClip Data Outliers:")
print(clip_outlier_results)

#### Outliers for X_Val

##### Mendeteksi dan menangani outliers

In [ ]:
# Cek box plot for all numerical data (X_val)

cols_per_row = 4
num_cols = X_val_numerical_all.shape[1]
num_rows = (num_cols + cols_per_row - 1) // cols_per_row

fig, axes = plt.subplots(nrows=num_rows, ncols=cols_per_row, figsize=(16, 4 * num_rows))
axes = axes.flatten()

for i, column in enumerate(X_val_numerical_all.columns):
    X_val_numerical_all.boxplot(column=column, ax=axes[i])
    axes[i].set_title(f'Box Plot for {column}')
    axes[i].set_ylabel('Values')

for j in range(i + 1, len(axes)):
    axes[j].set_visible(False)

plt.tight_layout()
plt.show()

#### Handling Outliers

##### Logform for data in a range 0-1000

In [ ]:
X_val_numerical_filtered = X_val_numerical_all.loc[:, (X_val_numerical_all.max() <= 1000) & (X_val_numerical_all.min() >= 0)]

X_val_log_transformed_filtered = np.log1p(X_val_numerical_filtered)

cols_per_row = 4
num_cols = X_val_log_transformed_filtered.shape[1]
num_rows = (num_cols + cols_per_row - 1) // cols_per_row

fig, axes = plt.subplots(nrows=num_rows, ncols=cols_per_row, figsize=(16, 4 * num_rows))
axes = axes.flatten()

for i, column in enumerate(X_val_log_transformed_filtered.columns):
    X_val_log_transformed_filtered.boxplot(column=column, ax=axes[i])
    axes[i].set_title(f'Log Transformed Box Plot for {column}')
    axes[i].set_ylabel('Log Values')

for j in range(i + 1, len(axes)):
    axes[j].set_visible(False)

plt.tight_layout()
plt.show()

##### SQRT untuk Data dengan rentang terdekat

In [ ]:
X_val_small_range = X_val_numerical_all.loc[:, (X_val_numerical_all.max() <= 15) & (X_val_numerical_all.min() >= 0)]

X_val_sqrt_transformed = np.sqrt(X_val_small_range)

cols_per_row = 4
num_cols = X_val_sqrt_transformed.shape[1]
num_rows = (num_cols + cols_per_row - 1) // cols_per_row

fig, axes = plt.subplots(nrows=num_rows, ncols=cols_per_row, figsize=(16, 4 * num_rows))
axes = axes.flatten()

for i, column in enumerate(X_val_sqrt_transformed.columns):
    X_val_sqrt_transformed.boxplot(column=column, ax=axes[i])
    axes[i].set_title(f'Sqrt Transformed Box Plot for {column}')
    axes[i].set_ylabel('Sqrt Values')

for j in range(i + 1, len(axes)):
    axes[j].set_visible(False)

plt.tight_layout()
plt.show()

##### Clip

#### Mendeteksi outliers after handling

In [ ]:
# X_val_numerical_all_clipped = clip__handlers_outliers(X_val_numerical_all)
# print("\nClipped Data Head:")
# print(X_val_numerical_all_clipped.head())

##### After Logtransform

In [ ]:
log_outlier_results = detect_outliers(X_val_log_transformed_filtered)
print("Log-Transformed Data Outliers:")
print(log_outlier_results)

##### After SQRT

In [ ]:
# Check outliers in sqrt-transformed data
sqrt_outlier_results = detect_outliers(X_val_sqrt_transformed)
print("\nSqrt-Transformed Data Outliers:")
print(sqrt_outlier_results)

##### After Clip

In [ ]:
# # Check outliers in sqrt-transformed data
# clip_outlier_results = detect_outliers(X_val_numerical_all_clipped)
# print("\nClip Data Outliers:")
# print(clip_outlier_results)

In [ ]:
X_val.head()

### III. Data Validation
Check for data integrity and consistency. Ensure that data types are correct, categorical variables have consistent labels, and numerical values fall within expected ranges.

In [ ]:
def data_validation(X):

  # Data Types Validation
  validate = X.copy()
  # Change all 'object' features into 'string'
  validate = validate.apply(lambda col: col.astype('string') if col.dtypes == 'object' else col)


  # Change all categorical 'float64' into 'int'
  columns_to_validate = categorical_columns_filtered + discrete_columns
  validate[columns_to_validate] = validate[columns_to_validate].apply(lambda col: col.astype('int'))


  # Binary categorical features validation (must contains 1 ror 0)
  # classes_validate_dict_validate = {col: validate[col].unique() for col in categorical_columns_filtered}
  return validate


# # FINAL:
# # Change all 'object' features into 'string'
# X_train = X_train.apply(lambda col: col.astype('string') if col.dtypes == 'object' else col)
# X_val = X_val.apply(lambda col: col.astype('string') if col.dtypes == 'object' else col)

# # Change all categorical 'float64' into 'int'
# columns_to_validate = categorical_columns_filtered + discrete_columns
# X_train[columns_to_validate] = X_train[columns_to_validate].apply(lambda col: col.astype('int'))
# X_val[columns_to_validate] = X_val[columns_to_validate].apply(lambda col: col.astype('int'))

# # Binary categorical features validation (must contains 1 ror 0)
# classes_X_train_dict_validate = {col: X_train[col].unique() for col in categorical_columns_filtered}
# classes_X_val_dict_validate = {col: X_val[col].unique() for col in categorical_columns_filtered}

# ==============

# # TEST:
# X_train_validation_test = X_train.copy()
# X_val_validation_test = X_val.copy()

# # Change all 'object' features into 'string'
# X_train_validation_test = X_train_validation_test.apply(lambda col: col.astype('string') if col.dtypes == 'object' else col)
# X_val_validation_test = X_val_validation_test.apply(lambda col: col.astype('string') if col.dtypes == 'object' else col)

# # Change all categorical 'float64' into 'int'
# columns_to_validate = categorical_columns_filtered + discrete_columns
# X_train_validation_test[columns_to_validate] = X_train_validation_test[columns_to_validate].apply(lambda col: col.astype('int'))
# # X_val_validation_test[columns_to_validate] = X_val_validation_test[columns_to_validate].apply(lambda col: col.astype('int'))

# # Binary categorical features validation (must contains 1 ror 0)
# classes_X_train_dict_validate_test = {col: X_train_validation_test[col].unique() for col in categorical_columns_filtered}
# classes_X_val_dict_validate_test = {col: X_val_validation_test[col].unique() for col in categorical_columns_filtered}

In [ ]:
# FINAL:
print("All Data Types of X_train:\n")
print(f"Rows (Data): {X_train.size}")
print(f"Columns (Features): {X_train.columns.size}")
X_train.dtypes

# ==============

# # TEST:
# print("All Data Types of X_val_validation_test:\n")
# print(f"Rows (Data): {X_val_validation_test.size}")
# print(f"Columns (Features): {X_val_validation_test.columns.size}")
# # print(f"All Categorical features of X_val_validation_test contains: {classes_X_val_dict_validate_test}\n")
# X_val_validation_test.dtypes

In [ ]:
# FINAL:
print("All Data Types of X_val:\n")
print(f"Rows (Data): {X_val.size}")
print(f"Columns (Features): {X_val.columns.size}")
X_val.dtypes

# ==============

# # TEST:
# print("All Data Types of X_val_validation_test:\n")
# print(f"Rows (Data): {X_val_validation_test.size}")
# print(f"Columns (Features): {X_val_validation_test.columns.size}")
# # print(f"All Categorical features of X_val_validation_test contains: {classes_X_val_dict_validate_test}\n")
# X_val_validation_test.dtypes

### IV. Remove Duplicates
Handling duplicate values is crucial because they can compromise data integrity, leading to inaccurate analysis and insights. Duplicate entries can bias machine learning models, causing overfitting and reducing their ability to generalize to new data. They also inflate the dataset size unnecessarily, increasing computational costs and processing times. Additionally, duplicates can distort statistical measures and lead to inconsistencies, ultimately affecting the reliability of data-driven decisions and reporting. Ensuring data quality by removing duplicates is essential for accurate, efficient, and consistent analysis.

In [ ]:
num_duplicates = X_train.duplicated().sum()
print(f"Number of duplicated columns: {num_duplicates}")

X_train = X_train.drop_duplicates()

print(f"Jumlah baris setelah menghapus duplikat: {X_train.shape[0]}")
print(f"Jumlah kolom: {X_train.shape[1]}")

# print("Data X_train setelah menghapus duplikat:")
# display(X_train.style.set_table_styles(
#     [{'selector': 'thead th', 'props': [('background-color', '#f7f7f7'), ('font-weight', 'bold')]},
#      {'selector': 'tbody td', 'props': [('border', '1px solid #ddd')]}]
# ).set_properties(**{'text-align': 'center'}))

### V. Feature Engineering

**Feature engineering** involves creating new features (input variables) or transforming existing ones to improve the performance of machine learning models. Feature engineering aims to enhance the model's ability to learn patterns and make accurate predictions from the data. It's often said that "good features make good models."

1. **Feature Selection:** Feature engineering can involve selecting the most relevant and informative features from the dataset. Removing irrelevant or redundant features not only simplifies the model but also reduces the risk of overfitting.

2. **Creating New Features:** Sometimes, the existing features may not capture the underlying patterns effectively. In such cases, engineers create new features that provide additional information. For example:
   
   - **Polynomial Features:** Engineers may create new features by taking the square, cube, or other higher-order terms of existing numerical features. This can help capture nonlinear relationships.
   
   - **Interaction Features:** Interaction features are created by combining two or more existing features. For example, if you have features "length" and "width," you can create an "area" feature by multiplying them.

3. **Binning or Discretization:** Continuous numerical features can be divided into bins or categories. For instance, age values can be grouped into bins like "child," "adult," and "senior."

4. **Domain-Specific Feature Engineering:** Depending on the domain and problem, engineers may create domain-specific features. For example, in fraud detection, features related to transaction history and user behavior may be engineered to identify anomalies.

Feature engineering is both a creative and iterative process. It requires a deep understanding of the data, domain knowledge, and experimentation to determine which features will enhance the model's predictive power.

#### V. 1: Feature Selection

In [ ]:
def feature_selection(X, y):
    # 1. Feature Selection

    # Chi2 Selector and Anova Selector
    chi2_selector = SelectKBest(score_func=chi2, k=20)
    anova_selector = SelectKBest(score_func=f_classif, k=15)

    # a) Fit kedua selector dengan data & label
    chi2_selector.fit(X[categorical_columns_filtered], y)
    anova_selector.fit(X[numerical_columns], y)

    # b) Ambil nama kolom terpilih
    selected_columns_chi2 = X[categorical_columns_filtered].columns[chi2_selector.get_support()]
    selected_columns_anova = X[numerical_columns].columns[anova_selector.get_support()]

    # c) Gabungkan kolom terpilih
    selected_cols_union = selected_columns_chi2.union(selected_columns_anova)
    X_selected = X[selected_cols_union]

    print(f"Selected Features: {X_selected.columns.tolist()}")

    return X_selected

X_train_feature_selection = X_train.copy()
y_train_feature_selection = y_train.copy()

X_val_feature_selection = X_val.copy()
y_val_feature_selection = y_val.copy()

feature_selection(X_train_feature_selection, y_train_feature_selection)
feature_selection(X_val_feature_selection, y_val_feature_selection)

#### V. 2: New Features

In [ ]:
# 2. New Features
def new_features(X):
  # a) NoOfPops -> Sum(NoOfPopup, NoOfiFrame)
  X['NoOfPops'] = X['NoOfPopup'] + X['NoOfiFrame']

  # b) NoOfHref -> Sum(NoOfSelfRef, NoOfEmptyRef, NoOfExternalRef)
  X['NoOfHref'] = X['NoOfSelfRef'] + X['NoOfEmptyRef'] + X['NoOfExternalRef']

  # c) HasSolidInfo -> HasCopyrightInfo || HasSocialNet
  X['HasSolidInfo'] = ((X['HasCopyrightInfo'] == 1) | (X['HasSocialNet'] == 1)).astype(int)

## B. Data Preprocessing

**Data preprocessing** is a broader step that encompasses both data cleaning and additional transformations to make the data suitable for machine learning algorithms. Its primary goals are:

1. **Feature Scaling:** Ensure that numerical features have similar scales. Common techniques include Min-Max scaling (scaling to a specific range) or standardization (mean-centered, unit variance).

2. **Encoding Categorical Variables:** Machine learning models typically work with numerical data, so categorical variables need to be encoded. This can be done using one-hot encoding, label encoding, or more advanced methods like target encoding.

3. **Handling Imbalanced Classes:** If dealing with imbalanced classes in a binary classification task, apply techniques such as oversampling, undersampling, or using different evaluation metrics to address class imbalance.

4. **Dimensionality Reduction:** Reduce the number of features using techniques like Principal Component Analysis (PCA) or feature selection to simplify the model and potentially improve its performance.

5. **Normalization:** Normalize data to achieve a standard distribution. This is particularly important for algorithms that assume normally distributed data.

### Notes on Preprocessing processes

It is advised to create functions or classes that have the same/similar type of inputs and outputs, so you can add, remove, or swap the order of the processes easily. You can implement the functions or classes by yourself

or

use `sklearn` library. To create a new preprocessing component in `sklearn`, implement a corresponding class that includes:
1. Inheritance to `BaseEstimator` and `TransformerMixin`
2. The method `fit`
3. The method `transform`

### I. Feature Scaling

**Feature scaling** is a preprocessing technique used in machine learning to standardize the range of independent variables or features of data. The primary goal of feature scaling is to ensure that all features contribute equally to the training process and that machine learning algorithms can work effectively with the data.

Here are the main reasons why feature scaling is important:

1. **Algorithm Sensitivity:** Many machine learning algorithms are sensitive to the scale of input features. If the scales of features are significantly different, some algorithms may perform poorly or take much longer to converge.

2. **Distance-Based Algorithms:** Algorithms that rely on distances or similarities between data points, such as k-nearest neighbors (KNN) and support vector machines (SVM), can be influenced by feature scales. Features with larger scales may dominate the distance calculations.

3. **Regularization:** Regularization techniques, like L1 (Lasso) and L2 (Ridge) regularization, add penalty terms based on feature coefficients. Scaling ensures that all features are treated equally in the regularization process.

Common methods for feature scaling include:

1. **Min-Max Scaling (Normalization):** This method scales features to a specific range, typically [0, 1]. It's done using the following formula:

   $$X' = \frac{X - X_{min}}{X_{max} - X_{min}}$$

   - Here, $X$ is the original feature value, $X_{min}$ is the minimum value of the feature, and $X_{max}$ is the maximum value of the feature.  
<br />
<br />
2. **Standardization (Z-score Scaling):** This method scales features to have a mean (average) of 0 and a standard deviation of 1. It's done using the following formula:

   $$X' = \frac{X - \mu}{\sigma}$$

   - $X$ is the original feature value, $\mu$ is the mean of the feature, and $\sigma$ is the standard deviation of the feature.  
<br />
<br />
3. **Robust Scaling:** Robust scaling is a method that scales features to the interquartile range (IQR) and is less affected by outliers. It's calculated as:

   $$X' = \frac{X - Q1}{Q3 - Q1}$$

   - $X$ is the original feature value, $Q1$ is the first quartile (25th percentile), and $Q3$ is the third quartile (75th percentile) of the feature.  
<br />
<br />
4. **Log Transformation:** In cases where data is highly skewed or has a heavy-tailed distribution, taking the logarithm of the feature values can help stabilize the variance and improve scaling.

The choice of scaling method depends on the characteristics of your data and the requirements of your machine learning algorithm. **Min-max scaling and standardization are the most commonly used techniques and work well for many datasets.**

Scaling should be applied separately to each training and test set to prevent data leakage from the test set into the training set. Additionally, **some algorithms may not require feature scaling, particularly tree-based models.**

In [ ]:
from sklearn.preprocessing import MinMaxScaler, StandardScaler, RobustScaler

In [ ]:
# X_train_filtered = X_train.copy().drop(columns=['TLD', 'Domain', 'URL', 'Title'])
X_train_scaled = X_train.copy()
numerical_columns = [col for col in X_train_scaled.columns if col not in categorical_columns]

#### Min Max

In [ ]:
def apply_minmax_scaling(data):
    min_max_scaler = MinMaxScaler()
    data[numerical_columns] = min_max_scaler.fit_transform(data[numerical_columns])
    return data

X_train_minmax = apply_minmax_scaling(X_train_scaled)
print("\nX_train_scaled after Min-Max Scaling:")
print(X_train_minmax)

#### Robust

In [ ]:
def apply_robust_scaling(data):
    robust_scaler = RobustScaler()
    data[numerical_columns] = robust_scaler.fit_transform(data[numerical_columns])
    return data

X_train_robust = apply_robust_scaling(X_train_scaled)
print("\nX_train_scaled after Robust Scaling:")
print(X_train_robust)

#### Standardization

In [ ]:
def apply_standardization(data):
    standard_scaler = StandardScaler()
    data[numerical_columns] = standard_scaler.fit_transform(data[numerical_columns])
    return data

X_train_standard = apply_standardization(X_train_scaled)
print("\nX_train_scaled after Standardization:")
print(X_train_standard)


X_train_scaled after Standardization:
                                      URL  URLLength  \
28021               https://www.yc-tp.com  -0.340615   
71026                                 NaN  -0.137735   
89927          https://www.autodesk.co.uk  -0.137735   
94654        https://www.marketmanila.com  -0.036295   
35275           https://www.re-course.org  -0.137735   
...                                   ...        ...   
110268           https://www.gemathis.com  -0.239175   
119879                                NaN  -0.188455   
103694                                NaN   0.318746   
131932       https://www.leinsterrugby.ie  -0.036295   
121958  https://www.cruiseharbournews.com   0.268026   

                           Domain  DomainLength  IsDomainIP  TLD  \
28021               www.yc-tp.com     -1.167639         0.0  com   
71026                         NaN     -0.283333         0.0  com   
89927          www.autodesk.co.uk     -0.283333         0.0   uk   
94654        www

### II. Feature Encoding

**Feature encoding**, also known as **categorical encoding**, is the process of converting categorical data (non-numeric data) into a numerical format so that it can be used as input for machine learning algorithms. Most machine learning models require numerical data for training and prediction, so feature encoding is a critical step in data preprocessing.

Categorical data can take various forms, including:

1. **Nominal Data:** Categories with no intrinsic order, like colors or country names.  

2. **Ordinal Data:** Categories with a meaningful order but not necessarily equidistant, like education levels (e.g., "high school," "bachelor's," "master's").

There are several common methods for encoding categorical data:

1. **Label Encoding:**

   - Label encoding assigns a unique integer to each category in a feature.
   - It's suitable for ordinal data where there's a clear order among categories.
   - For example, if you have an "education" feature with values "high school," "bachelor's," and "master's," you can encode them as 0, 1, and 2, respectively.
<br />
<br />
2. **One-Hot Encoding:**

   - One-hot encoding creates a binary (0 or 1) column for each category in a nominal feature.
   - It's suitable for nominal data where there's no inherent order among categories.
   - Each category becomes a new feature, and the presence (1) or absence (0) of a category is indicated for each row.
<br />
<br />
3. **Target Encoding (Mean Encoding):**

   - Target encoding replaces each category with the mean of the target variable for that category.
   - It's often used for classification problems.

In [ ]:
# Not needed, since the data is already properly encoded

### III. Handling Imbalanced Dataset

**Handling imbalanced datasets** is important because imbalanced data can lead to several issues that negatively impact the performance and reliability of machine learning models. Here are some key reasons:

1. **Biased Model Performance**:

 - Models trained on imbalanced data tend to be biased towards the majority class, leading to poor performance on the minority class. This can result in misleading accuracy metrics.

2. **Misleading Accuracy**:

 - High overall accuracy can be misleading in imbalanced datasets. For example, if 95% of the data belongs to one class, a model that always predicts the majority class will have 95% accuracy but will fail to identify the minority class.

3. **Poor Generalization**:

 - Models trained on imbalanced data may not generalize well to new, unseen data, especially if the minority class is underrepresented.


Some methods to handle imbalanced datasets:
1. **Resampling Methods**:

 - Oversampling: Increase the number of instances in the minority class by duplicating or generating synthetic samples (e.g., SMOTE).
 - Undersampling: Reduce the number of instances in the majority class to balance the dataset.

2. **Evaluation Metrics**:

 - Use appropriate evaluation metrics such as precision, recall, F1-score, ROC-AUC, and confusion matrix instead of accuracy to better assess model performance on imbalanced data.

3. **Algorithmic Approaches**:

 - Use algorithms that are designed to handle imbalanced data, such as decision trees, random forests, or ensemble methods.
 - Adjust class weights in algorithms to give more importance to the minority class.

In [ ]:
class_distribution = y_train.value_counts()
print("Class distribution:")
print(class_distribution)

class_distribution.plot(kind='bar', color='#ADD8E6')
plt.title('Class Distribution')
plt.xlabel('Class')
plt.ylabel('Frequency')
plt.show()

#### Oversampling with SMOTE

In [ ]:
smote = SMOTE(random_state=42)
X_train_balanced, y_train_balanced = smote.fit_resample(X_train_numerical_all, y_train)

balanced_class_distribution = pd.Series(y_train_balanced).value_counts()
print("Balanced class distribution:")
print(balanced_class_distribution)

In [ ]:
balanced_class_distribution.plot(kind='bar', color='#ADD8E6')
plt.title('Balanced Class Distribution')
plt.xlabel('Class')
plt.ylabel('Frequency')
plt.show()

print(f"Shape of X_train_balanced: {X_train_balanced.shape}")
print(f"Shape of y_train_balanced: {y_train_balanced.shape}")

print("Sample of resampled X_train:")
print(X_train_balanced[:5])

# 3. Compile Preprocessing Pipeline

All of the preprocessing classes or functions defined earlier will be compiled in this step.

If you use sklearn to create preprocessing classes, you can list your preprocessing classes in the Pipeline object sequentially, and then fit and transform your data.

In [ ]:
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler

# DataCleaning Transformer
class DataPreparation(BaseEstimator, TransformerMixin):
    def __init__(self):
        return None


    def fit(self, X, y=None):
        return self

    def transform(self, X):
        categorical_columns = [
            'IsDomainIP', 'HasObfuscation', 'IsHTTPS', 'HasTitle', 'HasFavicon',
            'Robots', 'IsResponsive', 'HasDescription', 'HasSocialNet',
            'HasSubmitButton', 'HasHiddenFields', 'HasPasswordField', 'Bank',
            'Pay', 'Crypto', 'HasCopyrightInfo', 'Domain', 'URL',
            'TLD', 'Title', 'HasExternalFormSubmit','NoOfSelfRedirect', 'NoOfURLRedirect'
        ]

        categorical_columns_to_fill = [
            'IsDomainIP', 'IsHTTPS', 'HasTitle', 'IsResponsive', 'Pay', 'HasHiddenFields', 'Robots',
            'Crypto', 'HasDescription', 'Bank', 'HasExternalFormSubmit', 'HasFavicon',
            'HasSubmitButton', 'HasPasswordField', 'NoOfSelfRedirect', 'HasCopyrightInfo',
            'NoOfURLRedirect', 'HasSocialNet'
        ]
        numerical_columns = [col for col in X.columns if col not in categorical_columns]
        tld_prob_mean = X.groupby('TLD')['TLDLegitimateProb'].mean()
        global_fill_value = X['TLDLegitimateProb'].median()
        group_by_columns = ['HasObfuscation']

        X = fill_url_length(X, url_col='URL', url_length_col='URLLength')
        X = fill_domain(X, url_col='URL', domain_col='Domain')
        X = fill_domain_length(X, domain_col='Domain', domain_length_col='DomainLength')
        X = fill_is_domain_ip(X, domain_col='Domain', is_domain_ip_col='IsDomainIP')
        X = fill_tld(X, domain_col='Domain', tld_col='TLD')
        X = fill_char_continuation_rate(X, url_col='URL', char_rate_col='CharContinuationRate')
        X = fill_tld_legitimate_prob(X, tld_col='TLD', tld_prob_col='TLDLegitimateProb', is_domain_ip_col='IsDomainIP')
        char_prob = calculate_char_prob(X, 'URL')
        X = fill_url_char_prob(X, url_col='URL', char_prob_col='URLCharProb', char_prob=char_prob)
        X = fill_tld_length(X, tld_col='TLD', tld_length_col='TLDLength')
        X = fill_no_of_subdomains(X, domain_col='Domain', subdomain_col='NoOfSubDomain')
        X = fill_has_obfuscation(X, url_col='URL', obfuscation_col='HasObfuscation')
        X = fill_no_of_obfuscated_characters(X, url_col='URL', obf_char_col='NoOfObfuscatedChar')
        X = fill_obfuscation_ratio(X, obf_char_col='NoOfObfuscatedChar', url_length_col='URLLength', obf_ratio_col='ObfuscationRatio')
        X = fill_no_of_letters_in_url(X, url_col='URL', letters_col='NoOfLettersInURL')
        X = fill_letter_ratio_in_url(X, letters_col='NoOfLettersInURL', url_length_col='URLLength', ratio_col='LetterRatioInURL')
        X = fill_no_of_digits_in_url(X, url_col='URL', digits_col='NoOfDegitsInURL')
        X = fill_digits_ratio_in_url(X, url_col='URL', ratio_col='DegitRatioInURL')
        X = fill_no_of_equals_in_url(X, url_col='URL', equals_col='NoOfEqualsInURL')
        X = fill_no_of_qmark_in_url(X, url_col='URL', qmark_col='NoOfQMarkInURL')
        X = fill_no_of_ampersand_in_url(X, url_col='URL', ampersand_col='NoOfAmpersandInURL')
        X = fill_no_of_special_chars_in_url(X, url_col='URL', special_chars_col='NoOfOtherSpecialCharsInURL')
        X = fill_specials_ratio_in_url(X, url_col='URL', ratio_col='SpacialCharRatioInURL')
        X = fill_is_https(X, url_col='URL', https_col='IsHTTPS')
        X = fill_has_title(X, title_col='Title', has_title_col='HasTitle')
        X = fill_domain_title_match_score(X,
            title_col='Title',
            domain_col='Domain',
            score_col='DomainTitleMatchScore'
        )
        X = fill_url_title_match_score(
            X,
            title_col='Title',
            url_col='URL',
            score_col='URLTitleMatchScore'
        )

        X = fill_numerical_columns(X, numerical_columns, skewness_threshold=3)
        X = fill_categorical_columns(X, categorical_columns_to_fill, group_by_columns)
        X = outlier_handlers_clip(X, lower_percentile=0.01, upper_percentile=0.99)
        X = X.drop(['URL', 'Domain', 'TLD', 'Title'], axis=1)
        X = data_validation(X)

        return X


pipeline = Pipeline([
    ('data_preparation', DataPreparation())
])

# Apply Pipeline

X_train_processed = pipeline.fit_transform(X_train_pipe_test)
X_val_processed = pipeline.transform(X_val_pipe_test)

In [ ]:
X_train_processed.isnull().sum()

## Load Test Data

In [ ]:
df_test = pd.read_csv('https://files.faizath.com/test.csv', sep=',', low_memory=False)

In [ ]:
id_test = df_test["id"]
df_test = df_test.drop(['id', 'FILENAME'], axis=1)
pipeline_test = Pipeline([
    ('data_preparation', DataPreparation())
])
X_test = pipeline_test.fit_transform(df_test)

Uses the mean for imputation because of the normal distribution.
Global fill value: 0.2570590781017271
Column URLLength: Using median for imputation because distribution is highly skewed.
Column DomainLength: Using mean for imputation because distribution is approximately normal.
Column CharContinuationRate: Using mean for imputation because distribution is approximately normal.
Column TLDLegitimateProb: Using mean for imputation because distribution is approximately normal.
Column URLCharProb: Using mean for imputation because distribution is approximately normal.
Column TLDLength: Using mean for imputation because distribution is approximately normal.
Column NoOfSubDomain: Using mean for imputation because distribution is approximately normal.
Column NoOfObfuscatedChar: Using median for imputation because distribution is highly skewed.
Column ObfuscationRatio: Using median for imputation because distribution is highly skewed.
Column NoOfLettersInURL: Using median for imputation becau

# 4. Modeling and Validation

Modelling is the process of building your own machine learning models to solve specific problems, or in this assignment context, predicting the target feature `label`. Validation is the process of evaluating your trained model using the validation set or cross-validation method and providing some metrics that can help you decide what to do in the next iteration of development.

In [ ]:
import pickle
from datetime import datetime
from zoneinfo import ZoneInfo

def date():
  return datetime.now(tz=ZoneInfo("Asia/Jakarta"))

## A. KNN

#### KNN from Scratch

In [ ]:
# Scale for KNN Only

X_train_processed_scaled = X_train_processed.copy()
X_val_processed_scaled = X_val_processed.copy()

# X_train_processed_scaled = X_train_processed_scaled.drop(['URL', 'Domain', 'TLD', 'Title'], axis=1)
# X_val_processed_scaled = X_val_processed_scaled.drop(['URL', 'Domain', 'TLD', 'Title'], axis=1)

X_train_processed_scaled = apply_standardization(X_train_processed)
X_val_processed_scaled = apply_standardization(X_val_processed)

In [ ]:
from collections import Counter

class KNN:
    def __init__(self, k=20, metric='euclidean', p=1):
        if k < 1 or not isinstance(k, int):
            raise ValueError("Invalid neighbor count. k must be an integer greater than 0.")
        if metric not in ['manhattan', 'euclidean', 'minkowski']:
            raise ValueError("Invalid distance metric. Valid metric: 'euclidean', 'manhattan', or 'minkowski'.")
        if p < 1 or not isinstance(p, int):
            raise ValueError("Invalid minkowski distance variable. p must be an integer greater than 0.")

        self.target = 'label'
        self.k = k
        self.metric = metric
        self.p = p

    def fit(self, X_train, y_train):
        self.X_train = X_train
        self.y_train = y_train

    def predict(self, X_test):
        if self.X_train is None or self.y_train is None:
            raise Exception("Invalid data")

        predictions = []
        for i, row in X_test.iterrows():
            match self.metric:
              case 'euclidean':
                  distances = np.sqrt(np.sum(np.square(row[np.array(X_test.columns)] - self.X_train), axis=1))
              case 'manhattan':
                  distances =  np.sum(np.abs(row[np.array(X_test.columns)] - self.X_train), axis=1)
              case 'minkowski':
                  distances = np.sum(np.abs(row[np.array(X_test.columns)] - self.X_train) ** self.p, axis=1) ** (1 / self.p)
              case _:
                  raise ValueError("Invalid distance metric. Valid metric: 'euclidean', 'manhattan', or 'minkowski'.")

            # Get the k nearest neighbors
            neighbors = np.argsort(distances)[:self.k]

            # Get the most common class
            classes = Counter(self.y_train.iloc[neighbors])
            max_class = classes.most_common(1)[0][0]

            predictions.append([i, max_class])
        return pd.DataFrame(predictions, columns=['id', self.target])

In [ ]:
def date():
  return datetime.now(tz=ZoneInfo("Asia/Jakarta"))

knn_val = KNN(k=20)
knn_val.fit(X_train_processed_scaled.head(10000), y_train)

start_time = date()
print("Start Time:", start_time.strftime("%H:%M:%S"))

knn_val_pred = knn_val.predict(X_val_processed_scaled)

end_time = date()
difference = end_time - start_time
print("Start Time:", start_time.strftime("%H:%M:%S"))
print("End Time:", end_time.strftime("%H:%M:%S"))
print("Time Elapsed:", difference)

Start Time: 22:48:57
Start Time: 22:48:57
End Time: 22:54:40
Time Elapsed: 0:05:42.922906


In [ ]:
# Evaluasi
# accuracy = accuracy_score(y_val, knn_pred['label'])
# precision = precision_score(y_val, knn_pred['label'], average='weighted')
# recall = recall_score(y_val, knn_pred['label'], average='weighted')

accuracy_knn_scratch = accuracy_score(y_val, knn_pred['label'])
report_knn_scratch = classification_report(y_val, knn_pred['label'])
cm_knn_scratch = confusion_matrix(y_val, knn_pred['label'])

print("====== KNN from Scratch =====")
print("Akurasi KNN         :", accuracy_knn_scratch)
print("\nClassification Report (KNN):")
print(report_knn_scratch)
print("Confusion Matrix (KNN):")
print(cm_knn_scratch)

           id  label
0        8409      1
1       20959      1
2       88797      1
3       24345      1
4       95019      1
...       ...    ...
28076   36268      1
28077   39295      1
28078  111590      1
28079  134414      1
28080   37313      1

[28081 rows x 2 columns]
------------------------------------------------
Accuracy: 0.9806986930664863
Precision: 0.9806612735137247
Recall: 0.9806986930664863


In [ ]:
from collections import Counter

class KNNTest:
    def __init__(self, k=20, metric='euclidean', p=1):
        if k < 1 or not isinstance(k, int):
            raise ValueError("Invalid neighbor count. k must be an integer greater than 0.")
        if metric not in ['manhattan', 'euclidean', 'minkowski']:
            raise ValueError("Invalid distance metric. Valid metric: 'euclidean', 'manhattan', or 'minkowski'.")
        if p < 1 or not isinstance(p, int):
            raise ValueError("Invalid minkowski distance variable. p must be an integer greater than 0.")

        self.target = 'label'
        self.k = k
        self.metric = metric
        self.p = p

    def fit(self, X_train, y_train):
        self.X_train = X_train
        self.y_train = y_train

    def predict(self, X_test):
        if self.X_train is None or self.y_train is None:
            raise Exception("Invalid data")

        predictions = []
        for i, row in X_test.iterrows():
            match self.metric:
              case 'euclidean':
                  distances = np.sqrt(np.sum(np.square(row[np.array(X_test.columns)] - self.X_train), axis=1))
              case 'manhattan':
                  distances =  np.sum(np.abs(row[np.array(X_test.columns)] - self.X_train), axis=1)
              case 'minkowski':
                  distances = np.sum(np.abs(row[np.array(X_test.columns)] - self.X_train) ** self.p, axis=1) ** (1 / self.p)
              case _:
                  raise ValueError("Invalid distance metric. Valid metric: 'euclidean', 'manhattan', or 'minkowski'.")

            # Get the k nearest neighbors
            neighbors = np.argsort(distances)[:self.k]

            # Get the most common class
            classes = Counter(self.y_train.iloc[neighbors])
            max_class = classes.most_common(1)[0][0]

            predictions.append([id_test.iloc[i], max_class])
        return pd.DataFrame(predictions, columns=['id', self.target])

In [ ]:
def date():
  return datetime.now(tz=ZoneInfo("Asia/Jakarta"))

knn_test = KNNTest(k=20)
knn_test.fit(X_train_processed_scaled, y_train)

start_time = date()
print("Start Time:", start_time.strftime("%H:%M:%S"))

knn_test_pred = knn_test.predict(X_test)

end_time = date()
difference = end_time - start_time
print("Start Time:", start_time.strftime("%H:%M:%S"))
print("End Time:", end_time.strftime("%H:%M:%S"))
print("Time Elapsed:", difference)

Start Time: 23:15:54
Start Time: 23:15:54
End Time: 23:29:45
Time Elapsed: 0:13:50.714329


In [ ]:
knn_test_pred.to_csv("test_dump.csv", sep=',', encoding='utf-8', index=False)

#### KNN by Scikit-Learn

In [ ]:
# KNN by Scikit-Learn
knn_scikit = KNeighborsClassifier(n_neighbors=6)

knn_scikit.fit(X_train_processed_scaled.head(10000), y_train.head(10000))

# Prediksi
start_time = date()
print("Start Time:", start_time.strftime("%H:%M:%S"))

y_pred_knn_scikit = knn_scikit.predict(X_val_processed_scaled)

end_time = date()
difference = end_time - start_time
print("Start Time:", start_time.strftime("%H:%M:%S"))
print("End Time:", end_time.strftime("%H:%M:%S"))
print("Time Elapsed:", difference)

Start Time: 23:45:52
Start Time: 23:45:52
End Time: 23:46:09
Time Elapsed: 0:00:16.603162


In [ ]:
# Evaluasi
accuracy_knn = accuracy_score(y_val, y_pred_knn_scikit)
report_knn = classification_report(y_val, y_pred_knn_scikit)
cm_knn = confusion_matrix(y_val, y_pred_knn_scikit)

print("====== KNN by Scikit-Learn =====")
print("Akurasi KNN         :", accuracy_knn)
print("\nClassification Report (KNN):")
print(report_knn)
print("Confusion Matrix (KNN):")
print(cm_knn)

====== KNN by Scikit-Learn =====
Akurasi KNN         : 0.9864321071186923

Classification Report (KNN):
              precision    recall  f1-score   support

           0       0.96      0.86      0.91      2152
           1       0.99      1.00      0.99     25929

    accuracy                           0.99     28081
   macro avg       0.98      0.93      0.95     28081
weighted avg       0.99      0.99      0.99     28081

Confusion Matrix (KNN):
[[ 1844   308]
 [   73 25856]]


In [ ]:
# KNN by Scikit-Learn
knn_scikit_test = KNeighborsClassifier(n_neighbors=6)

knn_scikit_test.fit(X_train_processed_scaled, y_train)

# Prediksi
start_time = date()
print("Start Time:", start_time.strftime("%H:%M:%S"))

y_pred_knn_scikit_test = knn_scikit_test.predict(X_test)

end_time = date()
difference = end_time - start_time
print("Start Time:", start_time.strftime("%H:%M:%S"))
print("End Time:", end_time.strftime("%H:%M:%S"))
print("Time Elapsed:", difference)

Start Time: 23:47:48
Start Time: 23:47:48
End Time: 23:48:02
Time Elapsed: 0:00:14.074040


In [ ]:
predictions = []
for i, row in enumerate(y_pred_knn_scikit_test):
  predictions.append([id_test.iloc[i], row])
pd.DataFrame(predictions, columns=['id', "label"]).to_csv("knn_dump.csv", sep=',', encoding='utf-8', index=False)

## B. Naive Bayes

In [ ]:
import math

class NaiveBayes:
    def __init__(self):
        self.target = 'label'
        self.pv = {}
        self.table = {}

    def _calculate_probability(self, x, mean, std):
        if std == 0:
          std = 1e-10
        exponent = -((x - mean) ** 2) / (2 * std ** 2)
        coefficient = 1 / (math.sqrt(2 * math.pi) * std)
        probability = coefficient * math.exp(exponent)
        return probability

    def fit(self, X_train, y_train):
        self.X_train = X_train
        self.y_train = y_train

        for i in range(2):
            self.pv[i] = len(y_train[y_train == i]) / len(y_train)

        data = X_train.join(y_train)

        for feature in np.array(X_train.columns):
            for k in range(2):
                k_data = data[data[self.target] == k]
                if not k_data.empty:
                    mean = k_data[feature].mean()
                    std = k_data[feature].std()
                    self.table[(feature, k)] = {'mean': mean, 'std': std}

    def predict(self, X_test):
        predictions = []
        for i, row in X_test.iterrows():
            class_probability = []
            for k in range(2):
                p = self.pv[k]
                for feature in X_test.columns:
                    if feature in np.array(self.X_train.columns):
                        value = row[feature]
                        mean = self.table[(feature, k)]['mean']
                        std = self.table[(feature, k)]['std']
                        p *= self._calculate_probability(value, mean, std)
                    else:
                        value = row[feature]
                        if (feature, value, k) in self.table:
                            p *= self.table[(feature, value, k)]

                class_probability.append(p)
            predictions.append([i, np.argmax(class_probability)])
        return pd.DataFrame(predictions, columns=['id', self.target])

    def __str__(self):
        string = "P(V):\n"
        for key, value in self.pv.items():
            string += f"P(V{key}): {value}\n"
        string += "Table:\n"
        for key, value in self.table.items():
            string += f"P({key[0]} | {key[1]}): {value}\n"
        return string

In [ ]:
def naive_bayes_drop(X, y):
  nb_df = X.join(y)
  f_drop = []
  for f in categorical_columns_filtered:
    for i, j in [(x//2, x%2) for x in range(4)]:
        if len(nb_df[(nb_df[f] == i) & (nb_df["label"] == j)]) == 0:
          f_drop.append(f)
  for i in X.columns:
    if X[i].std() == 0:
      f_drop.append(i)
  return f_drop

In [ ]:
nb_drop = naive_bayes_drop(X_train_processed_scaled, y_train)
nb_train = X_train_processed_scaled.drop(nb_drop, axis=1)
nb_val = X_val_processed_scaled.drop(nb_drop, axis=1)
nb_val_test = X_test.drop(nb_drop, axis=1)

In [ ]:
# Evaluasi
accuracy_nb_scratch = accuracy_score(y_val, nb_pred['label'])
report_knn_scratch = classification_report(y_val, nb_pred['label'])
cm_knn_scratch = confusion_matrix(y_val, nb_pred['label'])

print("====== KNN from Scratch =====")
print("Akurasi KNN         :", accuracy_knn)
print("\nClassification Report (KNN):")
print(report_knn)
print("Confusion Matrix (KNN):")
print(cm_knn)

Start Time: 23:38:37
Start Time: 23:38:37
End Time: 23:39:39
Time Elapsed: 0:01:01.762985


#### Gaussian Naive Bayes from Scratch

In [ ]:
nb = NaiveBayes()
nb.fit(nb_test, y_train)

start_time = date()
print("Start Time:", start_time.strftime("%H:%M:%S"))

nb_pred = nb.predict(nb_val)

end_time = date()
difference = end_time - start_time
print("Start Time:", start_time.strftime("%H:%M:%S"))
print("End Time:", end_time.strftime("%H:%M:%S"))
print("Time Elapsed:", difference)

In [ ]:
# Evaluasi
accuracy_nb_scratch = accuracy_score(y_val, nb_pred['label'])
report_knn_scratch = classification_report(y_val, nb_pred['label'])
cm_knn_scratch = confusion_matrix(y_val, nb_pred['label'])

print("====== Gaussian Naive Bayes from Scratch =====")
print("Akurasi NB         :", accuracy_knn)
print("\nClassification Report (NB):")
print(report_knn)
print("Confusion Matrix (NB):")
print(cm_knn)

In [ ]:
print(nb_pred)
accuracy = accuracy_score(y_val, nb_pred['label'])
precision = precision_score(y_val, nb_pred['label'], average='weighted')
recall = recall_score(y_val, nb_pred['label'], average='weighted')

print("------------------------------------------------")
print(f"Accuracy: {accuracy}")
print(f"Precision: {precision}")
print(f"Recall: {recall}")

           id  label
0        8409      1
1       20959      1
2       88797      1
3       24345      1
4       95019      1
...       ...    ...
28076   36268      1
28077   39295      1
28078  111590      1
28079  134414      1
28080   37313      1

[28081 rows x 2 columns]
------------------------------------------------
Accuracy: 0.9543107439193761
Precision: 0.9681675614798116
Recall: 0.9543107439193761


In [ ]:
import math

class NaiveBayesTest:
    def __init__(self):
        self.target = 'label'
        self.pv = {}
        self.table = {}

    def _calculate_probability(self, x, mean, std):
        if std == 0:
          std = 1e-10
        exponent = -((x - mean) ** 2) / (2 * std ** 2)
        coefficient = 1 / (math.sqrt(2 * math.pi) * std)
        probability = coefficient * math.exp(exponent)
        return probability

    def fit(self, X_train, y_train):
        self.X_train = X_train
        self.y_train = y_train

        for i in range(2):
            self.pv[i] = len(y_train[y_train == i]) / len(y_train)

        data = X_train.join(y_train)

        for feature in np.array(X_train.columns):
            for k in range(2):
                k_data = data[data[self.target] == k]
                if not k_data.empty:
                    mean = k_data[feature].mean()
                    std = k_data[feature].std()
                    self.table[(feature, k)] = {'mean': mean, 'std': std}

    def predict(self, X_test):
        predictions = []
        for i, row in X_test.iterrows():
            class_probability = []
            for k in range(2):
                p = self.pv[k]
                for feature in X_test.columns:
                    if feature in np.array(self.X_train.columns):
                        value = row[feature]
                        mean = self.table[(feature, k)]['mean']
                        std = self.table[(feature, k)]['std']
                        p *= self._calculate_probability(value, mean, std)
                    else:
                        value = row[feature]
                        if (feature, value, k) in self.table:
                            p *= self.table[(feature, value, k)]

                class_probability.append(p)
            predictions.append([id_test.iloc[i], np.argmax(class_probability)])
        return pd.DataFrame(predictions, columns=['id', self.target])

    def __str__(self):
        string = "P(V):\n"
        for key, value in self.pv.items():
            string += f"P(V{key}): {value}\n"
        string += "Table:\n"
        for key, value in self.table.items():
            string += f"P({key[0]} | {key[1]}): {value}\n"
        return string

In [ ]:
def date():
  return datetime.now(tz=ZoneInfo("Asia/Jakarta"))

nbtest = NaiveBayesTest()
nbtest.fit(nb_train, y_train)

start_time = date()
print("Start Time:", start_time.strftime("%H:%M:%S"))

nb_test_pred = nbtest.predict(nb_val_test)

end_time = date()
difference = end_time - start_time
print("Start Time:", start_time.strftime("%H:%M:%S"))
print("End Time:", end_time.strftime("%H:%M:%S"))
print("Time Elapsed:", difference)

Start Time: 23:34:08
Start Time: 23:34:08
End Time: 23:34:30
Time Elapsed: 0:00:22.219472


In [ ]:
nb_test_pred.to_csv("nb_test_dump.csv", sep=',', encoding='utf-8', index=False)

#### Gaussian Naive Bayes by Scikit-Learn

In [ ]:
# Guassian NB by Scikit-Learn
model = GaussianNB()

model.fit(nb_test, y_train)

# Prediksi
y_pred = model.predict(nb_val)

start_time = date()
print("Start Time:", start_time.strftime("%H:%M:%S"))

y_pred_knn_scikit = knn_scikit.predict(X_val_processed_scaled)

end_time = date()
difference = end_time - start_time
print("Start Time:", start_time.strftime("%H:%M:%S"))
print("End Time:", end_time.strftime("%H:%M:%S"))
print("Time Elapsed:", difference)

In [ ]:
# Evaluasi
accuracy_nb = accuracy_score(y_val, y_pred)
report_nb = classification_report(y_val, y_pred)
cm_nb = confusion_matrix(y_val, y_pred)

print("====== Gaussian Naive Bayes from Scratch =====")
print("Akurasi           :", accuracy_nb)
print("\nClassification Report:")
print(report_nb)
print("Confusion Matrix:")
print(cm_nb)

## C. Submission
To predict the test set target feature and submit the results to the kaggle competition platform, do the following:
1. Create a new pipeline instance identical to the first in Data Preprocessing
2. With the pipeline, apply `fit_transform` to the original training set before splitting, then only apply `transform` to the test set.
3. Retrain the model on the preprocessed training set
4. Predict the test set
5. Make sure the submission contains the `id` and `label` column.

Note: Adjust step 1 and 2 to your implementation of the preprocessing step if you don't use pipeline API from `sklearn`.

In [ ]:
df_test = pd.read_csv('https://files.faizath.com/test.csv', sep=',', low_memory=False)

In [ ]:
id_test = df_test["id"]
df_test = df_test.drop(['id', 'FILENAME'], axis=1)
pipeline_test = Pipeline([
    ('data_preparation', DataPreparation())
])
X_test = pipeline_test.fit_transform(df_test)

# 6. Error Analysis

Based on all the process you have done until the modeling and evaluation step, write an analysis to support each steps you have taken to solve this problem. Write the analysis using the markdown block. Some questions that may help you in writing the analysis:

- Does my model perform better in predicting one class than the other? If so, why is that?
- To each models I have tried, which performs the best and what could be the reason?
- Is it better for me to impute or drop the missing data? Why?
- Does feature scaling help improve my model performance?
- etc...

**Analisis Model**
---

Penanganan Missing Data
---
Pada tahap data cleaning, data yang hilang diisi menggunakan strategi berbasis domain sesuai dengan sifat fitur yang relevan terhadap fitur lain. Untuk fitur numerik yang terdistribusi normal, imputasi dilakukan menggunakan mean. Sementara itu, untuk fitur numerik yang tidak terdistribusi normal, digunakan median, dan modus dengan teknik groupby diterapkan untuk fitur kategorikal. Pendekatan ini memastikan bahwa informasi yang hilang dapat diisi tanpa mengorbankan kontribusi penting pada algoritma berbasis jarak seperti KNN dan Gaussian Naive Bayes.


Scaling untuk Meningkatkan Performa
---
Feature scaling dilakukan untuk memastikan performa optimal karena algoritma KNN sangat bergantung pada jarak antar data. Pada proses ini, fitur numerik dinormalisasi menggunakan metode z-score normalization. Hal ini membantu memastikan bahwa tidak ada fitur yang mendominasi perhitungan jarak akibat skala yang berbeda, sehingga model mampu menangkap pola dengan lebih baik dan memberikan hasil prediksi yang lebih akurat.


Performa Model KNN dengan Parameter K yang optimal
---
Pada eksperimen yang dilakukan, nilai k=20 dipilih sebagai parameter terbaik untuk algoritma KNN. Penggunaan nilai k=20 dengan metrik jarak Euclidean memberikan hasil terbaik karena metrik ini lebih cocok untuk menangkap pola data tabular yang telah dinormalisasi dengan baik. Model dengan k=20 mampu menghasilkan prediksi yang cukup stabil dan tidak terlalu lambat karena tidak semua fitur berguna untuk dibaca dari keseluruhan fitur yaitu 30 fitur yang terseleksi.

Saran Perbaikan
---
- Pertimbangkan penggunaan algoritma alternatif seperti Random Forest atau Gradient Boosting untuk menangani ketidakseimbangan data secara lebih efektif.
- Gunakan teknik oversampling tambahan untuk lebih meningkatkan distribusi data kelas minoritas, atau eksplorasi pengaturan ulang bobot pada algoritma KNN (Weighted KNN).
-Tambahkan fitur baru yang lebih relevan untuk membantu model mengenali pola pada data minoritas (creating new features).
